In [ ]:
import random
from tqdm import tqdm
from pathlib import Path
import matplotlib as mpl
import plotly.graph_objects as go
#random.seed(99)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt,six
import scipy
from scipy.stats.stats import pearsonr
import dask.dataframe as dd
import seaborn as sns
import dask.datasets
from dask_sql import Context
import datetime
import time
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None
import findspark
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ArrayType, BooleanType
from pyspark.sql.functions import udf, coalesce, col, lit, desc
import pyspark.sql.functions as f
import docx
from docx.shared import Cm
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import pandas as pd
from matplotlib.ticker import StrMethodFormatter, FuncFormatter

import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn import preprocessing
import matplotlib as mpl
from svgpathtools import svg2paths
from svgpath2mpl import parse_path
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import PIL
from PIL import Image
import docx
from docx.enum.dml import MSO_THEME_COLOR_INDEX

In [4]:
def row_number(df, col, sort_col, index, desc=True, filter_df=True, filter_zero=False):
    if col not in df.columns:
        df[col] = ''
    if filter_df:
        rows_to_count = df[df[col]].sort_values(sort_col,ascending=not desc).loc[:,[col]].index[:index]
    elif filter_zero:
        rows_to_count = df[df[col]!=0].sort_values(sort_col,ascending=not desc).loc[:,[col]].index[:index]
    else:
        rows_to_count = df.sort_values(sort_col,ascending=not desc).loc[:,[col]].index[:index]
        df_shape = df.shape
    rows_to_zero = [i for i in df.index if i not in rows_to_count] 
    n = 0
    for r0 in rows_to_zero:
        df.loc[r0, col] = 0
    for r in rows_to_count:
        n += 1
        df.loc[r, col] = n

    return df

def count_ratings(df, index, ratings_to_count):
# Пользователи
    df_user_max = df.loc[:,['Пользователи 2020','Пользователи 2021','Пользователи 2022','Пользователи 2023']].max(axis=1)
    df_user_min = df.loc[:,['Пользователи 2020','Пользователи 2021','Пользователи 2022','Пользователи 2023']].min(axis=1)

    #df['Рейтинг по пользователям 2020'] = df_user_max == df.loc[:,['Пользователи 2020']].iloc[:,0]
    #df['Рейтинг по пользователям 2021'] = df_user_max == df.loc[:,['Пользователи 2021']].iloc[:,0]
    #df['Рейтинг по пользователям 2022'] = df_user_max == df.loc[:,['Пользователи 2022']].iloc[:,0]
    #df['Рейтинг по пользователям 2023'] = df_user_max == df.loc[:,['Пользователи 2023']].iloc[:,0]

    df = row_number(df, 'Рейтинг по пользователям 2020', 'Пользователи 2020', index=index, filter_df=False)
    df = row_number(df, 'Рейтинг по пользователям 2021', 'Пользователи 2021', index=index, filter_df=False)
    df = row_number(df, 'Рейтинг по пользователям 2022', 'Пользователи 2022', index=index, filter_df=False)
    df = row_number(df, 'Рейтинг по пользователям 2023', 'Пользователи 2023', index=index, filter_df=False)
    df = row_number(df, 'Рейтинг по пользователям,\nитого', 'Пользователи, итого', index=index, filter_df=False)
    
    
    #df['Антирейтинг по пользователям 2020'] = df_user_min == df.loc[:,['Пользователи 2020']].iloc[:,0]
    #df['Антирейтинг по пользователям 2021'] = df_user_min == df.loc[:,['Пользователи 2021']].iloc[:,0]
    #df['Антирейтинг по пользователям 2022'] = df_user_min == df.loc[:,['Пользователи 2022']].iloc[:,0]
    #df['Антирейтинг по пользователям 2023'] = df_user_min == df.loc[:,['Пользователи 2023']].iloc[:,0]

    df = row_number(df, 'Антирейтинг по пользователям 2020', 'Пользователи 2020', desc=False, index=index, filter_df=False)
    df = row_number(df, 'Антирейтинг по пользователям 2021', 'Пользователи 2021', desc=False, index=index, filter_df=False)
    df = row_number(df, 'Антирейтинг по пользователям 2022', 'Пользователи 2022', desc=False, index=index, filter_df=False)
    df = row_number(df, 'Антирейтинг по пользователям 2023', 'Пользователи 2023', desc=False, index=index, filter_df=False)
    df = row_number(df, 'Антирейтинг по пользователям,\nитого', 'Пользователи, итого', desc=False, index=index, filter_df=False)
    
    
    df['Рейтинг постоянного роста\nпользователей 2020-2023'] = ((df['Пользователи 2020'] < df['Пользователи 2021']) & (df['Пользователи 2021'] < df['Пользователи 2022']) & (df['Пользователи 2022'] < df['Пользователи 2023']))
    df['Рейтинг постоянного роста\nпользователей 2021-2023'] = ((df['Пользователи 2021'] < df['Пользователи 2022']) & (df['Пользователи 2022'] < df['Пользователи 2023']))
    df['Рейтинг постоянного роста\nпользователей 2022-2023'] = ((df['Пользователи 2022'] < df['Пользователи 2023']))
    
    df = row_number(df, 'Рейтинг постоянного роста\nпользователей 2020-2023', 'Пользователи, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного роста\nпользователей 2021-2023', 'Пользователи, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного роста\nпользователей 2022-2023', 'Пользователи, итого', index=index)
    
    
    df['Рейтинг постоянного падения\nпользователей 2020-2023'] = ((df['Пользователи 2020'] > df['Пользователи 2021'])  & (df['Пользователи 2021'] > df['Пользователи 2022']) & (df['Пользователи 2022'] > df['Пользователи 2023']))
    df['Рейтинг постоянного падения\nпользователей 2021-2023'] = ((df['Пользователи 2021'] > df['Пользователи 2022']) & (df['Пользователи 2022'] > df['Пользователи 2023']))
    df['Рейтинг постоянного падения\nпользователей 2022-2023'] = ((df['Пользователи 2022'] > df['Пользователи 2023']))
    
    df = row_number(df, 'Рейтинг постоянного падения\nпользователей 2020-2023', 'Пользователи, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного падения\nпользователей 2021-2023', 'Пользователи, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного падения\nпользователей 2022-2023', 'Пользователи, итого', index=index)

    df['Рейтинг по пользователям НННВ'] = ((df['Пользователи 2020'] < df['Пользователи 2023'])  & (df['Пользователи 2021'] < df['Пользователи 2023']) & (df['Пользователи 2022'] < df['Пользователи 2023']))
    df['Рейтинг по пользователям ННВВ'] = ((df['Пользователи 2020'] < df['Пользователи 2022'])  & (df['Пользователи 2020'] < df['Пользователи 2023']) & (df['Пользователи 2021'] < df['Пользователи 2022']) & (df['Пользователи 2021'] < df['Пользователи 2023']))
    df['Рейтинг по пользователям НВВВ'] = ((df['Пользователи 2020'] < df['Пользователи 2021'])  & (df['Пользователи 2020'] < df['Пользователи 2022']) & (df['Пользователи 2020'] < df['Пользователи 2023']))
    df['Рейтинг по пользователям ВВВН'] = ((df['Пользователи 2023'] < df['Пользователи 2020'])  & (df['Пользователи 2023'] < df['Пользователи 2022']) & (df['Пользователи 2023'] < df['Пользователи 2021']))
    df['Рейтинг по пользователям ВВНН'] = ((df['Пользователи 2020'] > df['Пользователи 2022'])  & (df['Пользователи 2020'] > df['Пользователи 2023']) & (df['Пользователи 2021'] > df['Пользователи 2022']) & (df['Пользователи 2021'] > df['Пользователи 2023']))
    df['Рейтинг по пользователям ВННН'] = ((df['Пользователи 2020'] > df['Пользователи 2022'])  & (df['Пользователи 2020'] > df['Пользователи 2023']) & (df['Пользователи 2020'] > df['Пользователи 2021']))
    df['Рейтинг по пользователям ВНВН'] = ((df['Пользователи 2020'] > df['Пользователи 2021'])  & (df['Пользователи 2021'] < df['Пользователи 2022']) & (df['Пользователи 2022'] > df['Пользователи 2023']))
    df['Рейтинг по пользователям НВНВ'] = ((df['Пользователи 2020'] < df['Пользователи 2021'])  & (df['Пользователи 2021'] > df['Пользователи 2022']) & (df['Пользователи 2022'] < df['Пользователи 2023']))
    df['Рейтинг по пользователям ВННВ'] = ((df['Пользователи 2020'] > df['Пользователи 2021'])  & (df['Пользователи 2021'] < df['Пользователи 2022']) & (df['Пользователи 2022'] > df['Пользователи 2023']))
    df['Рейтинг по пользователям НВВН'] = ((df['Пользователи 2020'] < df['Пользователи 2021'])  & (df['Пользователи 2021'] > df['Пользователи 2022']) & (df['Пользователи 2022'] > df['Пользователи 2023']))

    df = row_number(df, 'Рейтинг по пользователям НННВ', 'Пользователи 2023', index=index)
    df = row_number(df, 'Рейтинг по пользователям ННВВ', 'Пользователи 2023', index=index)
    df = row_number(df, 'Рейтинг по пользователям НВВВ', 'Пользователи 2023', index=index)
    df = row_number(df, 'Рейтинг по пользователям ВВВН', 'Пользователи 2023', index=index)
    df = row_number(df, 'Рейтинг по пользователям ВВНН', 'Пользователи 2023', index=index)
    df = row_number(df, 'Рейтинг по пользователям ВННН', 'Пользователи 2023', index=index)
    df = row_number(df, 'Рейтинг по пользователям ВНВН', 'Пользователи 2023', index=index)
    df = row_number(df, 'Рейтинг по пользователям НВНВ', 'Пользователи 2023', index=index)
    df = row_number(df, 'Рейтинг по пользователям ВННВ', 'Пользователи 2023', index=index)
    df = row_number(df, 'Рейтинг по пользователям НВВН', 'Пользователи 2023', index=index)
        

    df['Рейтинг стабильности\nчисла пользователей'] = abs(df_user_min / df_user_max - 1)
    df = row_number(df, 'Рейтинг стабильности\nчисла пользователей', 'Пользователи, итого', filter_df=False, index=index)

    
# Запросы    
    df_req_max = df.loc[:,['Запросы 2020','Запросы 2021','Запросы 2022','Запросы 2023']].max(axis=1)
    df_req_min = df.loc[:,['Запросы 2020','Запросы 2021','Запросы 2022','Запросы 2023']].min(axis=1)
    #df['Рейтинг по запросам 2020'] = df_req_max == df.loc[:,['Запросы 2020']].iloc[:,0]
    #df['Рейтинг по запросам 2021'] = df_req_max == df.loc[:,['Запросы 2021']].iloc[:,0]
    #df['Рейтинг по запросам 2022'] = df_req_max == df.loc[:,['Запросы 2022']].iloc[:,0]
    #df['Рейтинг по запросам 2023'] = df_req_max == df.loc[:,['Запросы 2023']].iloc[:,0]
    
    df = row_number(df, 'Рейтинг по запросам 2020', 'Запросы 2020', index=index, filter_df=False)
    df = row_number(df, 'Рейтинг по запросам 2021', 'Запросы 2021', index=index, filter_df=False)
    df = row_number(df, 'Рейтинг по запросам 2022', 'Запросы 2022', index=index, filter_df=False)
    df = row_number(df, 'Рейтинг по запросам 2023', 'Запросы 2023', index=index, filter_df=False)
    df = row_number(df, 'Рейтинг по запросам, итого', 'Запросы, итого', filter_df=False, index=index)

    
    #df['Антирейтинг по запросам 2020'] = df_req_min == df.loc[:,['Запросы 2020']].iloc[:,0]
    #df['Антирейтинг по запросам 2021'] = df_req_min == df.loc[:,['Запросы 2021']].iloc[:,0]
    #df['Антирейтинг по запросам 2022'] = df_req_min == df.loc[:,['Запросы 2022']].iloc[:,0]
    #df['Антирейтинг по запросам 2023'] = df_req_min == df.loc[:,['Запросы 2023']].iloc[:,0]
    
    df = row_number(df, 'Антирейтинг по запросам 2020', 'Запросы 2020', desc=False, index=index, filter_df=False)
    df = row_number(df, 'Антирейтинг по запросам 2021', 'Запросы 2021', desc=False, index=index, filter_df=False)
    df = row_number(df, 'Антирейтинг по запросам 2022', 'Запросы 2022', desc=False, index=index, filter_df=False)
    df = row_number(df, 'Антирейтинг по запросам 2023', 'Запросы 2023', desc=False, index=index, filter_df=False)
    df = row_number(df, 'Антирейтинг по запросам, итого', 'Запросы, итого', desc=False, index=index, filter_df=False)

    
    df['Рейтинг постоянного роста\nзапросов 2020-2023'] = ((df['Запросы 2020'] < df['Запросы 2021']) & (df['Запросы 2021'] < df['Запросы 2022']) & (df['Запросы 2022'] < df['Запросы 2023']))
    df['Рейтинг постоянного роста\nзапросов 2021-2023'] = ((df['Запросы 2021'] < df['Запросы 2022']) & (df['Запросы 2022'] < df['Запросы 2023']))
    df['Рейтинг постоянного роста\nзапросов 2022-2023'] = ((df['Запросы 2022'] < df['Запросы 2023']))
    
    df = row_number(df, 'Рейтинг постоянного роста\nзапросов 2020-2023', 'Запросы, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного роста\nзапросов 2021-2023', 'Запросы, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного роста\nзапросов 2022-2023', 'Запросы, итого', index=index)
    
    
    df['Рейтинг постоянного падения\nзапросов 2020-2023'] = ((df['Запросы 2020'] > df['Запросы 2021'])  & (df['Запросы 2021'] > df['Запросы 2022']) & (df['Запросы 2022'] > df['Запросы 2023']))
    df['Рейтинг постоянного падения\nзапросов 2021-2023'] = ((df['Запросы 2021'] > df['Запросы 2022']) & (df['Запросы 2022'] > df['Запросы 2023']))
    df['Рейтинг постоянного падения\nзапросов 2022-2023'] = ((df['Запросы 2022'] > df['Запросы 2023']))
    
    df = row_number(df, 'Рейтинг постоянного падения\nзапросов 2020-2023', 'Запросы, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного падения\nзапросов 2021-2023', 'Запросы, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного падения\nзапросов 2022-2023', 'Запросы, итого', index=index)
    
    
    df['Рейтинг по запросам НННВ'] = ((df['Запросы 2020'] < df['Запросы 2023'])  & (df['Запросы 2021'] < df['Запросы 2023']) & (df['Запросы 2022'] < df['Запросы 2023']))
    df['Рейтинг по запросам ННВВ'] = ((df['Запросы 2020'] < df['Запросы 2022'])  & (df['Запросы 2020'] < df['Запросы 2023']) & (df['Запросы 2021'] < df['Запросы 2022']) & (df['Запросы 2021'] < df['Запросы 2023']))
    df['Рейтинг по запросам НВВВ'] = ((df['Запросы 2020'] < df['Запросы 2021'])  & (df['Запросы 2020'] < df['Запросы 2022']) & (df['Запросы 2020'] < df['Запросы 2023']))
    df['Рейтинг по запросам ВВВН'] = ((df['Запросы 2023'] < df['Запросы 2020'])  & (df['Запросы 2023'] < df['Запросы 2022']) & (df['Запросы 2023'] < df['Запросы 2021']))
    df['Рейтинг по запросам ВВНН'] = ((df['Запросы 2020'] > df['Запросы 2022'])  & (df['Запросы 2020'] > df['Запросы 2023']) & (df['Запросы 2021'] > df['Запросы 2022']) & (df['Запросы 2021'] > df['Запросы 2023']))
    df['Рейтинг по запросам ВННН'] = ((df['Запросы 2020'] > df['Запросы 2022'])  & (df['Запросы 2020'] > df['Запросы 2023']) & (df['Запросы 2020'] > df['Запросы 2021']))
    df['Рейтинг по запросам ВНВН'] = ((df['Запросы 2020'] > df['Запросы 2021']) & (df['Запросы 2021'] < df['Запросы 2022'])& (df['Запросы 2022'] > df['Запросы 2023']))
    df['Рейтинг по запросам НВНВ'] = ((df['Запросы 2020'] < df['Запросы 2021'])  & (df['Запросы 2021'] > df['Запросы 2022']) & (df['Запросы 2022'] < df['Запросы 2023']))
    df['Рейтинг по запросам ВННВ'] = ((df['Запросы 2020'] > df['Запросы 2021'])  & (df['Запросы 2021'] < df['Запросы 2022']) & (df['Запросы 2022'] > df['Запросы 2023']))
    df['Рейтинг по запросам НВВН'] = ((df['Запросы 2020'] < df['Запросы 2021'])  & (df['Запросы 2021'] > df['Запросы 2022']) & (df['Запросы 2022'] > df['Запросы 2023']))


    df = row_number(df, 'Рейтинг по запросам НННВ', 'Запросы 2023', index=index)
    df = row_number(df, 'Рейтинг по запросам ННВВ', 'Запросы 2023', index=index)
    df = row_number(df, 'Рейтинг по запросам НВВВ', 'Запросы 2023', index=index)
    df = row_number(df, 'Рейтинг по запросам ВВВН', 'Запросы 2023', index=index)
    df = row_number(df, 'Рейтинг по запросам ВВНН', 'Запросы 2023', index=index)
    df = row_number(df, 'Рейтинг по запросам ВННН', 'Запросы 2023', index=index)
    df = row_number(df, 'Рейтинг по запросам ВНВН', 'Запросы 2023', index=index)
    df = row_number(df, 'Рейтинг по запросам НВНВ', 'Запросы 2023', index=index)
    df = row_number(df, 'Рейтинг по запросам ВННВ', 'Запросы 2023', index=index)
    df = row_number(df, 'Рейтинг по запросам НВВН', 'Запросы 2023', index=index)
    
    df['Рейтинг стабильности\nчисла запросов'] = abs(df_req_min / df_req_max - 1)
    df = row_number(df, 'Рейтинг стабильности\nчисла запросов', 'Запросы, итого', filter_df=False, index=index)
    
    
# Запросов на пользователя
    df_req_user_max = df.loc[:,['Запросов на пользователя 2020','Запросов на пользователя 2021','Запросов на пользователя 2022','Запросов на пользователя 2023']].max(axis=1)
    df_req_user_min = df.loc[:,['Запросов на пользователя 2020','Запросов на пользователя 2021','Запросов на пользователя 2022','Запросов на пользователя 2023']].min(axis=1)

    df['Рейтинг запросов\nна пользователя 2020'] = df_req_user_max == df.loc[:,['Запросов на пользователя 2020']].iloc[:,0]
    df['Рейтинг запросов\nна пользователя 2021'] = df_req_user_max == df.loc[:,['Запросов на пользователя 2021']].iloc[:,0]
    df['Рейтинг запросов\nна пользователя 2022'] = df_req_user_max == df.loc[:,['Запросов на пользователя 2022']].iloc[:,0]
    df['Рейтинг запросов\nна пользователя 2023'] = df_req_user_max == df.loc[:,['Запросов на пользователя 2023']].iloc[:,0]

    df = row_number(df, 'Рейтинг запросов\nна пользователя 2020', 'Запросов на пользователя 2020', index=index)
    df = row_number(df, 'Рейтинг запросов\nна пользователя 2021', 'Запросов на пользователя 2021', index=index)
    df = row_number(df, 'Рейтинг запросов\nна пользователя 2022', 'Запросов на пользователя 2022', index=index)
    df = row_number(df, 'Рейтинг запросов\nна пользователя 2023', 'Запросов на пользователя 2023', index=index)
    df = row_number(df, 'Рейтинг запросов\nна пользователя, итого', 'Запросов на пользователя, итого', filter_df=False, index=index)


    df['Антирейтинг запросов на пользователя 2020'] = df_req_user_min == df.loc[:,['Запросов на пользователя 2020']].iloc[:,0]
    df['Антирейтинг запросов на пользователя 2021'] = df_req_user_min == df.loc[:,['Запросов на пользователя 2021']].iloc[:,0]
    df['Антирейтинг запросов на пользователя 2022'] = df_req_user_min == df.loc[:,['Запросов на пользователя 2022']].iloc[:,0]
    df['Антирейтинг запросов на пользователя 2023'] = df_req_user_min == df.loc[:,['Запросов на пользователя 2023']].iloc[:,0]

    df = row_number(df, 'Антирейтинг запросов на пользователя 2020', 'Запросов на пользователя 2020', desc=False, index=index)
    df = row_number(df, 'Антирейтинг запросов на пользователя 2021', 'Запросов на пользователя 2021', desc=False, index=index)
    df = row_number(df, 'Антирейтинг запросов на пользователя 2022', 'Запросов на пользователя 2022', desc=False, index=index)
    df = row_number(df, 'Антирейтинг запросов на пользователя 2023', 'Запросов на пользователя 2023', desc=False, index=index)
    df = row_number(df, 'Антирейтинг запросов на пользователя, итого', 'Запросов на пользователя, итого', desc=False, filter_df=False, index=index)


    df['Рейтинг постоянного роста\nзапросов на пользователя 2020-2023'] = ((df['Запросов на пользователя 2020'] < df['Запросов на пользователя 2021']) & (df['Запросов на пользователя 2021'] < df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2022'] < df['Запросов на пользователя 2023']))
    df['Рейтинг постоянного роста\nзапросов на пользователя 2021-2023'] = ((df['Запросов на пользователя 2021'] < df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2022'] < df['Запросов на пользователя 2023']))
    df['Рейтинг постоянного роста\nзапросов на пользователя 2022-2023'] = ((df['Запросов на пользователя 2022'] < df['Запросов на пользователя 2023']))

    df = row_number(df, 'Рейтинг постоянного роста\nзапросов на пользователя 2020-2023', 'Запросов на пользователя, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного роста\nзапросов на пользователя 2021-2023', 'Запросов на пользователя, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного роста\nзапросов на пользователя 2022-2023', 'Запросов на пользователя, итого', index=index)


    df['Рейтинг постоянного падения\nзапросов на пользователя 2020-2023'] = ((df['Запросов на пользователя 2020'] > df['Запросов на пользователя 2021'])  & (df['Запросов на пользователя 2021'] > df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2022'] > df['Запросов на пользователя 2023']))
    df['Рейтинг постоянного падения\nзапросов на пользователя 2021-2023'] = ((df['Запросов на пользователя 2021'] > df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2022'] > df['Запросов на пользователя 2023']))
    df['Рейтинг постоянного падения\nзапросов на пользователя 2022-2023'] = ((df['Запросов на пользователя 2022'] > df['Запросов на пользователя 2023']))

    df = row_number(df, 'Рейтинг постоянного падения\nзапросов на пользователя 2020-2023', 'Запросов на пользователя, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного падения\nзапросов на пользователя 2021-2023', 'Запросов на пользователя, итого', index=index)
    df = row_number(df, 'Рейтинг постоянного падения\nзапросов на пользователя 2022-2023', 'Запросов на пользователя, итого', index=index)

    
    df['Рейтинг запросов на пользователя НННВ'] = ((df['Запросов на пользователя 2020'] < df['Запросов на пользователя 2023'])  & (df['Запросов на пользователя 2021'] < df['Запросов на пользователя 2023']) & (df['Запросов на пользователя 2022'] < df['Запросов на пользователя 2023']))
    df['Рейтинг запросов на пользователя ННВВ'] = ((df['Запросов на пользователя 2020'] < df['Запросов на пользователя 2022'])  & (df['Запросов на пользователя 2020'] < df['Запросов на пользователя 2023']) & (df['Запросов на пользователя 2021'] < df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2021'] < df['Запросов на пользователя 2023']))
    df['Рейтинг запросов на пользователя НВВВ'] = ((df['Запросов на пользователя 2020'] < df['Запросов на пользователя 2021'])  & (df['Запросов на пользователя 2020'] < df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2020'] < df['Запросов на пользователя 2023']))
    df['Рейтинг запросов на пользователя ВВВН'] = ((df['Запросов на пользователя 2023'] < df['Запросов на пользователя 2020'])  & (df['Запросов на пользователя 2023'] < df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2023'] < df['Запросов на пользователя 2021']))
    df['Рейтинг запросов на пользователя ВВНН'] = ((df['Запросов на пользователя 2020'] > df['Запросов на пользователя 2022'])  & (df['Запросов на пользователя 2020'] > df['Запросов на пользователя 2023']) & (df['Запросов на пользователя 2021'] > df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2021'] > df['Запросов на пользователя 2023']))
    df['Рейтинг запросов на пользователя ВННН'] = ((df['Запросов на пользователя 2020'] > df['Запросов на пользователя 2022'])  & (df['Запросов на пользователя 2020'] > df['Запросов на пользователя 2023']) & (df['Запросов на пользователя 2020'] > df['Запросов на пользователя 2021']))
    df['Рейтинг запросов на пользователя ВНВН'] = ((df['Запросов на пользователя 2020'] > df['Запросов на пользователя 2021'])  & (df['Запросов на пользователя 2021'] < df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2022'] > df['Запросов на пользователя 2023']))
    df['Рейтинг запросов на пользователя НВНВ'] = ((df['Запросов на пользователя 2020'] < df['Запросов на пользователя 2021'])  & (df['Запросов на пользователя 2021'] > df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2022'] < df['Запросов на пользователя 2023']))    
    df['Рейтинг запросов на пользователя ВННВ'] = ((df['Запросов на пользователя 2020'] > df['Запросов на пользователя 2021'])  & (df['Запросов на пользователя 2021'] < df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2022'] > df['Запросов на пользователя 2023']))
    df['Рейтинг запросов на пользователя НВВН'] = ((df['Запросов на пользователя 2020'] < df['Запросов на пользователя 2021'])  & (df['Запросов на пользователя 2021'] > df['Запросов на пользователя 2022']) & (df['Запросов на пользователя 2022'] > df['Запросов на пользователя 2023']))


    df = row_number(df, 'Рейтинг запросов на пользователя НННВ', 'Запросов на пользователя 2023', index=index)
    df = row_number(df, 'Рейтинг запросов на пользователя ННВВ', 'Запросов на пользователя 2023', index=index)
    df = row_number(df, 'Рейтинг запросов на пользователя НВВВ', 'Запросов на пользователя 2023', index=index)
    df = row_number(df, 'Рейтинг запросов на пользователя ВВВН', 'Запросов на пользователя 2023', index=index)
    df = row_number(df, 'Рейтинг запросов на пользователя ВВНН', 'Запросов на пользователя 2023', index=index)
    df = row_number(df, 'Рейтинг запросов на пользователя ВННН', 'Запросов на пользователя 2023', index=index)
    df = row_number(df, 'Рейтинг запросов на пользователя ВНВН', 'Запросов на пользователя 2023', index=index)
    df = row_number(df, 'Рейтинг запросов на пользователя НВНВ', 'Запросов на пользователя 2023', index=index)
    df = row_number(df, 'Рейтинг запросов на пользователя ВННВ', 'Запросов на пользователя 2023', index=index)
    df = row_number(df, 'Рейтинг запросов на пользователя НВВН', 'Запросов на пользователя 2023', index=index)

    
    df['Рейтинг стабильности\nчисла запросов на пользователя'] = abs(df_user_min / df_user_max - 1)
    df = row_number(df, 'Рейтинг стабильности\nчисла запросов на пользователя', 'Запросов на пользователя, итого', filter_df=False, index=index)
    
    
#Рейтинг схожести динамики
    x = df[['Пользователи 2020','Пользователи 2021','Пользователи 2022','Пользователи 2023']]
    y = df[['Запросы 2020','Запросы 2021','Запросы 2022','Запросы 2023']]
    df = df.fillna(0)
    for row in df.index:
        df.loc[row, 'Рейтинг схожести динамики\nпользователей и запросов'] = scipy.stats.pearsonr(x.loc[row], y.loc[row])[0]
    df['Антирейтинг схожести динамики\nпользователей и запросов'] = df['Рейтинг схожести динамики\nпользователей и запросов']
    df = row_number(df, 'Рейтинг схожести динамики\nпользователей и запросов', 'Запросы, итого', filter_df=False, index=index)
    df = row_number(df, 'Антирейтинг схожести динамики\nпользователей и запросов', 'Запросы, итого', desc=False, filter_df=False, index=index)
    
    
    x = df[['Пользователи 2020','Пользователи 2021','Пользователи 2022','Пользователи 2023']]
    y = df[['Запросов на пользователя 2020','Запросов на пользователя 2021','Запросов на пользователя 2022','Запросов на пользователя 2023']]
    df = df.fillna(0)
    for row in df.index:
        df.loc[row, 'Рейтинг схожести динамики\nпользователей и запросов на пользователя'] = scipy.stats.pearsonr(x.loc[row], y.loc[row])[0]
    df['Антирейтинг схожести динамики\nпользователей и запросов на пользователя'] = df['Рейтинг схожести динамики\nпользователей и запросов на пользователя']
    df = row_number(df, 'Рейтинг схожести динамики\nпользователей и запросов на пользователя', 'Запросов на пользователя, итого', filter_df=False, index=index)
    df = row_number(df, 'Антирейтинг схожести динамики\nпользователей и запросов на пользователя', 'Запросов на пользователя, итого', desc=False, filter_df=False, index=index)
    
       
    x = df[['Запросы 2020','Запросы 2021','Запросы 2022','Запросы 2023']]
    y = df[['Запросов на пользователя 2020','Запросов на пользователя 2021','Запросов на пользователя 2022','Запросов на пользователя 2023']]
    df = df.fillna(0)
    for row in df.index:
        df.loc[row, 'Рейтинг схожести динамики\nзапросов и запросов на пользователя'] = scipy.stats.pearsonr(x.loc[row], y.loc[row])[0]
    df['Антирейтинг схожести динамики\nзапросов и запросов на пользователя'] = df['Рейтинг схожести динамики\nзапросов и запросов на пользователя']
    df = row_number(df, 'Рейтинг схожести динамики\nзапросов и запросов на пользователя', 'Запросов на пользователя, итого', filter_df=False, index=index)
    df = row_number(df, 'Антирейтинг схожести динамики\nзапросов и запросов на пользователя', 'Запросов на пользователя, итого', desc=False, filter_df=False, index=index)
    
    df = df.drop(['№', 'Пользователи 2020', 'Пользователи 2021',
           'Пользователи 2022', 'Пользователи 2023', 'Пользователи, итого',
           'Пользователи, %', 'Запросы 2020', 'Запросы 2021', 'Запросы 2022',
           'Запросы 2023', 'Запросы, итого', 'Запросы, %',
           'Запросов на пользователя 2020', 'Запросов на пользователя 2021',
           'Запросов на пользователя 2022', 'Запросов на пользователя 2023',
           'Запросов на пользователя, итого'], axis=1).set_index('Рубрика')
    if isinstance(ratings_to_count, str):
        ratings_to_count = df.columns
    return df.loc[:,ratings_to_count]

def horizontal_optimization(table_to_optimize, index):
    new_svod = table_to_optimize
    test_svod = new_svod.copy()
    index_set = set(new_svod.index)
    for _ in tqdm(range(len(index_set))):
        j = index_set.pop()
        cols = [i for i in test_svod.loc[j][test_svod.loc[j] > 0].sort_values(ascending=False).index if 'cnt' not in i]
        for i in cols:  
            cnt_horizontal = test_svod.loc[j].drop('cnt_horizontal').replace(range(1, index+1), 1).sum()
            cnt_vertical = test_svod[i].replace(range(1, index+1), 1).sum()
            if cnt_vertical > 15 and cnt_horizontal > 1:
                test_svod[i][j] = 0
                
        cols_set = set(cols)
        while cols_set:
            i = cols_set.pop()
            cnt_horizontal = test_svod.loc[j].drop('cnt_horizontal').replace(range(1, index+1), 1).sum()
            cnt_vertical = test_svod[i].replace(range(1, index+1), 1).sum()
            if cnt_vertical > 15 and cnt_horizontal > 1:
                test_svod[i][j] = 0 

    test_svod['cnt_horizontal'] = test_svod.drop('cnt_horizontal', axis=1).replace(range(1, index+1), 1).sum(axis=1)
    test_svod = test_svod.sort_values('cnt_horizontal', ascending=False)
    return test_svod

def vertical_optimization(table_to_optimize, original_table, optimization_level, index):
    df = table_to_optimize.copy()
    new_svod = original_table.copy()
    print('Вертикальная оптимизация')
    for i in tqdm(range(optimization_level)):
        for cat_to_optimize in df.index:
            # сейчас есть значение, превращаем в ноль
            col = df.drop('cnt_horizontal', axis=1).loc[cat_to_optimize].sort_values(ascending=False).index[0]

            # сейчас 0 возвращаем значение из бэкапа
            try:
                cat_to_reset = (
                     df
                     .loc[df.index != cat_to_optimize]
                     .loc[(new_svod.sort_index().loc[:, col] != df.sort_index().loc[:, col]) & (df[col] == 0)]
                     .loc[:,[col, 'cnt_horizontal']]
                     .sort_values([col, 'cnt_horizontal'], ascending=[False, True])
                     .index[0]
                )
                if df.loc[cat_to_reset, 'cnt_horizontal'] + 1 < df.loc[cat_to_optimize, 'cnt_horizontal']:
                    df.loc[cat_to_optimize, col] = 0
                    df.loc[cat_to_reset, col] = new_svod.loc[cat_to_reset, col]
                    df['cnt_horizontal'] = df.drop('cnt_horizontal', axis=1).replace(range(1, index+1), 1).sum(axis=1)
            except:
                pass
    return df

def optimize_df(df_to_optimize, index, cut_level, calc_cut_level=False, optimization_level=10, ratings_to_count=''):
    original_svod = df_to_optimize.copy().iloc[:cut_level]
    print(original_svod.shape)
    original_svod_sorted = (
        original_svod
        .fillna(0)
        .set_index('Рубрика')
    ).sort_values('Рубрика')
    original_svod_sorted = original_svod.loc[[not i.isnumeric() for i in original_svod['Рубрика']]]
    original_svod_counted = count_ratings(original_svod_sorted, index=index, ratings_to_count=ratings_to_count)
    original_svod_counted['cnt_horizontal'] = original_svod_counted.replace(range(1, index+1), 1).sum(axis=1)
    original_svod_counted = original_svod_counted.sort_values('cnt_horizontal', ascending=False)

    hor_optimized_svod = horizontal_optimization(original_svod_counted, index=index)
    vert_optimized_svod = vertical_optimization(hor_optimized_svod, original_svod_counted, optimization_level, index=index)
    
    optimized_svod = vert_optimized_svod.reset_index()
    optimized_svod['В рейтингах, раз'] = optimized_svod['cnt_horizontal']
    optimized_svod = optimized_svod.drop('cnt_horizontal', axis=1).sort_index()

    df = (pd.DataFrame(original_svod_counted.value_counts('cnt_horizontal').rename('Оригинальный файл'))
     .merge(pd.DataFrame(hor_optimized_svod.value_counts('cnt_horizontal').rename('Горизонтальная оптимизация')), left_index=True, right_index=True, how='outer')
     .merge(pd.DataFrame(vert_optimized_svod.value_counts('cnt_horizontal').rename('Вертикальная оптимизация')), left_index=True, right_index=True, how='outer')
     .fillna(0).astype(int))

    display(df)
    
    if calc_cut_level:
        return df
    return optimized_svod

def make_final(original_svod, optimized_svod):
    original_svod_sorted = (
        original_svod
        .fillna(0)
        .set_index('Рубрика')
    ).sort_values('Рубрика')
    original_svod_sorted = original_svod.loc[[not i.isnumeric() for i in original_svod['Рубрика']]]
    
    final_svod = original_svod_sorted.copy().reset_index()

    final_svod = final_svod[['№', 'Рубрика', 'Пользователи 2020', 'Пользователи 2021',
           'Пользователи 2022', 'Пользователи 2023', 'Пользователи, итого',
           'Пользователи, %', 'Запросы 2020', 'Запросы 2021', 'Запросы 2022',
           'Запросы 2023', 'Запросы, итого', 'Запросы, %',
           'Запросов на пользователя 2020', 'Запросов на пользователя 2021',
           'Запросов на пользователя 2022', 'Запросов на пользователя 2023',
           'Запросов на пользователя, итого']].sort_values('№')

    final_svod = final_svod.merge(optimized_svod, left_on='Рубрика', right_on='Рубрика')
    final_svod = final_svod.sort_index().sort_values('№')
    for i in optimized_svod.drop(['Рубрика', 'В рейтингах, раз'], axis=1).columns:
        final_svod = row_number(final_svod, i, i, desc=False, filter_df=False, index=15, filter_zero=True)
    final_svod['Рейтинг суммарный'] = final_svod[optimized_svod.columns].sum(axis=1)
    final_svod['Рейтинг средний'] = final_svod[optimized_svod.columns].mean(axis=1)
    final_svod['Вне рейтингов'] = final_svod['В рейтингах, раз'].fillna(0) == 0
    final_svod = row_number(final_svod, 'Вне рейтингов', 'Запросы, итого', desc=True, filter_df=False, index=15, filter_zero=True)

    return final_svod
    
def rename(df1_req_cnt, df1_user_cnt):
    pd_df1_req_cnt = pd.DataFrame(df1_req_cnt)
    pd_df1_user_cnt = pd.DataFrame(df1_user_cnt)
    df1_agg = pd_df1_req_cnt.merge(pd_df1_user_cnt, left_on='rubricName', right_on='rubricName')
    df1_agg = df1_agg.sort_values('date', ascending=False)
    #df1_agg['rubricName'] = df1_agg['rubricName_x']
    df1_agg = df1_agg.set_index('rubricName')[['date', 'user']].fillna(0).astype(int)
    return df1_agg

#Стартуем с 450 до 750 по 50 пока кол-во 4 не станет = 0
#От найденного значения идем вниз по 10, пока кол-во 4 не станет > 0

def find_cut_level(svod, index, cut_level_num=400, optimization_level=10):
    df = optimize_df(svod, index=index, cut_level=cut_level_num, calc_cut_level=True, optimization_level=optimization_level) 
    while df[df.index > 3]['Вертикальная оптимизация'].sum() > 0:
        cut_level_num += 50
        df = optimize_df(svod, index=index, cut_level=cut_level_num, calc_cut_level=True, optimization_level=optimization_level) 
        
    while df.iloc[3:, 2].sum() == 0 and df.loc[0][2] > 14:
        cut_level_num -= 10
        df = optimize_df(svod, index=index, cut_level=cut_level_num, calc_cut_level=True, optimization_level=optimization_level) 
        
        
    
def show_time(func):
    def wrapped(*args):
        start_dt = datetime.datetime.now()
        res = func(*args)
        end_dt = datetime.datetime.now()
        print('Start:'+' '*4, str(start_dt)[:-7])
        print('End:'+' '*6, str(end_dt)[:-7])
        print('Duration:'+' '*1, str(end_dt-start_dt)[:-7])
        return res
    return wrapped

In [ ]:
@show_time
def count_itogo_users(*files):
    file_path = "\\".join(files[0].split("\\")[:-1]) + "\\" + files[0].split("\\")[-2]
    d = {}
    for file_path in files:
        df = dd.read_csv(file_path, dtype={'user': 'object'}, on_bad_lines='skip')[['user', 'rubricName', 'date']]
        year = file_path.split('_')[-2]
        user_cnt_year = df.user.nunique().compute()
        d[year] = user_cnt_year
        print(year, user_cnt_year)
    df_list = [dd.read_csv(file_path, dtype={'user': 'object'}, on_bad_lines='skip')[['rubricName', 'user']] for file_path in files]
    df_all = dd.concat(df_list)
    cnt_user_total = df_all.user.nunique().compute()
    d['Итого'] = cnt_user_total
    print(cnt_user_total)
    cnt_df = pd.DataFrame([d], index=["Итого"])
    cnt_df.to_csv(f'{file_path}_cnt_user_itogo.csv', index=False)
    return cnt_df
        
def count_users_requests(file_path):
    try:
        cnt_user_grouped_year = dd.read_csv(f'{file_path}_count_users_requests.csv')
        print('read ' + f'{file_path}_count_users_requests.csv')
    except:
        year = file_path.split('_')[-2]
        requests_name = f'Запросы {year}'
        users_name = f'Пользователи {year}'
        df = dd.read_csv(file_path, dtype={'user': 'object'}, on_bad_lines='skip')[['user', 'rubricName', 'date']]
        cnt_user_grouped = df.groupby('rubricName').user.nunique().to_frame()
        cnt_req_grouped = df.groupby('rubricName').date.count().to_frame()
        cnt_user_grouped_year = (cnt_req_grouped
                      .merge(cnt_user_grouped, left_on='rubricName', right_on='rubricName')
                      .rename(columns={'date':requests_name, 'user':users_name})
                      .sort_values(requests_name, ascending=False)
                      .fillna(0)
                      .astype(int)
                      )
        cnt_user_grouped_year.to_csv(f'{file_path}_count_users_requests.csv', single_file=True)
    finally:
        df = dd.read_csv(f'{file_path}_count_users_requests.csv')

    return df


def count_total_users(*files):
    file_path = "\\".join(files[0][0].split("\\")[:-1]) + "\\" + files[0][0].split("\\")[-2]
    try:
        cnt_user_grouped_total = dd.read_csv("{}_count_total_users.csv".format(file_path))
        print('read ' + "{}_count_total_users.csv".format(file_path))
    except:
        df_list = [dd.read_csv(file_path, dtype={'user': 'object'}, on_bad_lines='skip')[['rubricName', 'user']] for file_path in files]
        df_all = dd.concat(df_list)
        cnt_user_grouped_total = df_all.groupby('rubricName').user.nunique().to_frame().rename(columns={'user':'Пользователи, итого'})
        cnt_user_grouped_total.to_csv(f'{file_path}_count_total_users.csv', single_file=True)
    finally:
        df = dd.read_csv("{}_count_total_users.csv".format(file_path))
    return df

#@show_time
def make_base_counted_file(*files):
    file_path = "\\".join(files[0].split("\\")[:3]) + "\\" + files[0].split("\\")[2]
    df = count_users_requests(files[0])
    for file in tqdm(files[1:]):
        df = df.merge(count_users_requests(file), left_on='rubricName', right_on='rubricName', how='outer')
    df = df.merge(count_total_users(files), left_on='rubricName', right_on='rubricName', how='outer')
    df.to_csv(f'{file_path}_master_file.csv', single_file=True, index=False)
    return df

from dask.diagnostics import ProgressBar
ProgressBar().register()

In [ ]:
def repack_file_old(file):
    start_t = time.time()
    file_name = str(file).replace('\\', '\\\\')
    print(file_name)
    df = dd.read_csv(file, dtype={'fl': 'object'})
    df = df[['rubricName','time','date','tx', 'user']]
    df['time'] = df['time'].apply(lambda x: x.split(' ')[1].split('.')[0], meta=('time', 'object'))
    df['user'] = df['user'].apply(hash, meta=('user', 'int64'))
    new_file_name = file_name.replace('.csv', '_repacked.csv')
    df.to_csv(new_file_name, single_file=True, index=False)
    end_t = time.time()
    print(end_t-start_t)

def repack_file(file_name):
    new_name = file_name.replace('\\', '\\\\').replace('.csv', '_repacked.csv')
    print(new_name)
    start_t = time.time()
    df_schema = StructType([StructField('user', StringType(), True), StructField('proj', StringType(), True), StructField('rubricName', StringType(), True), StructField('rubricID', StringType(), True), StructField('time', StringType(), True), StructField('date', StringType(), True), StructField('org', StringType(), True), StructField('branch', StringType(), True), StructField('prod', StringType(), True), StructField('tx', StringType(), True), StructField('fl', StringType(), True), StructField('devmod', StringType(), True), StructField('lat', StringType(), True), StructField('lon', StringType(), True)])
    df1 = spark.read.csv(file_name, header='true', schema=df_schema).selectExpr('rubricName','substr(time, 12, 8) as time','date','tx', 'hash(user) user').where('rubricName is not null')
    df1.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").mode('overwrite').csv(new_name)  
    end_t = time.time()
    print(end_t-start_t)

In [9]:
import string
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

def normalize_word(word):
    return (sorted([morph.normal_forms(str(i))[0].strip(string.punctuation + string.whitespace) for i in word.split(' ')]))

normalize_word_udf = udf(normalize_word, ArrayType(StringType()))

def check_stopwords(word, russian_stopwords = russian_stopwords):
    if word not in russian_stopwords and len(word)>1:
        for letter in word:
            if letter in string.punctuation or letter in string.digits:
                return 0
        return 1
    return 0

check_stopwords_udf = udf(check_stopwords, StringType())

def prepare_requests_text(name, rubrics):
    df = spark.read.parquet(f"files/df_filtered_{name}")

    df.createOrReplaceTempView('df')

    grouped_df = spark.sql('''

    select
        rubricName
        ,year
        ,tx
        ,cnt
        ,row_number() over(partition by rubricName ,year order by cnt desc) rn
    from (
        select
            rubricName
            ,year(date) year
            ,lower(tx) tx
            ,count(*) cnt
        from df
        group by 
            rubricName
            ,year(date)
            ,lower(tx)
            ) t
    ''')#.where('rn < 16')

    
    grouped_df.write.mode('overwrite').parquet(f'files/df_tx_grouped_{name}')

    
    spark.read.parquet(f"files/df_tx_grouped_{name}").where('tx is not null and year in (2020,2021,2022,2023)').write.mode('overwrite').parquet(f'files/df_tx_grouped_filtered_{name}')

    df_tx_grouped_normalize = (spark.read.parquet(f"files/df_tx_grouped_filtered_{name}")#.limit(3000)
                               .withColumn('new_tx',normalize_word_udf(col('tx')))
                        )

    
    df_tx_grouped_normalize.write.mode('overwrite').parquet(f'files/df_tx_grouped_normalize_{name}')

    df_tx_grouped_normalize_agg_str = (
        spark.read.parquet(f"files/df_tx_grouped_normalize_{name}")
        .withColumn("new_tx_str1", f.trim(f.concat_ws(" ",col("new_tx")))).withColumn("new_tx_str", f.expr('''BTRIM(new_tx_str1, '—!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~')'''))
        .where('coalesce(new_tx_str, "") !="" ')
        .groupBy(f.col("rubricName"),f.col("new_tx_str"),f.col("year")) 
        .agg(f.sum("cnt").alias("count"))
        .sort(desc('count'))
        #.show()
    )

    
    df_tx_grouped_normalize_agg_str.write.mode('overwrite').parquet(f'files/df_tx_grouped_normalize_agg_str_{name}')

    df_tx_str = spark.read.parquet(f"files/df_tx_grouped_normalize_agg_str_{name}")

    df_tx_str.createOrReplaceTempView('df_tx_str')

    df_tx_grouped_normalize_final_str = spark.sql('''
    with t as
        (
        select
            rubricName
            ,new_tx_str
            ,case when year=2020 then sum(count) else 0 end cnt_2020
            ,case when year=2021 then sum(count) else 0 end cnt_2021
            ,case when year=2022 then sum(count) else 0 end cnt_2022
            ,case when year=2023 then sum(count) else 0 end cnt_2023
        from df_tx_str
        group by 
            rubricName
            ,new_tx_str
            ,year
        )
    , t1 as 
        (        
        select
            rubricName
            ,new_tx_str
            ,max(cnt_2020) cnt_2020
            ,max(cnt_2021) cnt_2021
            ,max(cnt_2022) cnt_2022
            ,max(cnt_2023) cnt_2023
        from t
        group by new_tx_str, rubricName
        )
    , t2 as 
        (
        select
            'Топ-70 рубрик' as rubricName
            ,new_tx_str
            ,max(cnt_2020) cnt_2020
            ,max(cnt_2021) cnt_2021
            ,max(cnt_2022) cnt_2022
            ,max(cnt_2023) cnt_2023
        from t
        group by 'Топ-70 рубрик', new_tx_str
        )
    , t3 as 
        (
        select * from t1
        union all
        select * from t2
        )

    select *, cnt_2020+cnt_2021+cnt_2022+cnt_2023 as cnt_total, row_number() over(partition by rubricName order by cnt_2020+cnt_2021+cnt_2022+cnt_2023 desc) rn
    from t3

    ''').where('rn < 71')

    df_tx_grouped_normalize_final_str.write.mode('overwrite').parquet(f'files/df_tx_grouped_normalize_final_str_{name}')

    df_tx_str = (spark.read.parquet(f'files/df_tx_grouped_normalize_{name}').withColumn("new_tx_str1", f.trim(f.concat_ws(" ",col("new_tx")))).withColumn("new_tx_str", f.expr('''BTRIM(new_tx_str1, '—!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~')'''))
            .where('coalesce(new_tx_str, "") !="" and rn <= 100')
            .select('rubricName', 'tx', 'year', 'new_tx_str'))

    df_tx_str.createOrReplaceTempView('df_tx_str')

    df_tx_str_collect = spark.sql('''

    select 
        new_tx_str
        , rubricName
        , concat_ws('',slice(collect_set(tx),1,1)) tx_1
        , concat_ws('',slice(collect_set(tx),2,1)) tx_2
        , concat_ws('',slice(collect_set(tx),3,1)) tx_3
        , concat_ws('',slice(collect_set(tx),4,1)) tx_4
        , concat_ws('',slice(collect_set(tx),5,1)) tx_5
    from df_tx_str
    group by new_tx_str, rubricName

    ''')

    df_tx_str_collect.write.mode('overwrite').parquet(f'files/df_tx_str_collect_{name}')

    df_tx_grouped_normalize_final_str = spark.read.parquet(f"files/df_tx_grouped_normalize_final_str_{name}")
    df_tx_grouped_normalize_final_str.createOrReplaceTempView('df_tx_grouped_normalize_final_str')

    df_tx_str_collect = spark.read.parquet(f"files/df_tx_str_collect_{name}")
    df_tx_str_collect.createOrReplaceTempView('df_tx_str_collect')

    df_final = spark.sql('''

    select f.*,tx_1 ,tx_2 ,tx_3 ,tx_4 ,tx_5
    from df_tx_grouped_normalize_final_str f
    left join df_tx_str_collect c
    on f.rubricName = c.rubricName and f.new_tx_str = c.new_tx_str

    ''')

    df_final.write.mode('overwrite').parquet(f'files/df_final_{name}')

    pdf_str = spark.read.parquet(f'files/df_final_{name}').toPandas()

    cols = ['Рубрика', 'Поисковый запрос', 'Количество запросов 2020', 'Количество запросов 2021'
            , 'Количество запросов 2022', 'Количество запросов 2023', 'Количество запросов, итого'
            , 'Рейтинг по запросам, итого','tx_1' ,'tx_2' ,'tx_3' ,'tx_4' ,'tx_5']

    pdf_str.columns = cols

    rubrics = rubrics.copy()

    rubrics['Рейтинг рубрики'] = rubrics.index + 1
    cols.append('Рейтинг рубрики')

    pdf_new_str = rubrics.merge(pdf_str, how='outer')[cols]
    pdf_new_str.loc[pdf_new_str['Рубрика'] == 'Топ-70 рубрик', ['Рейтинг рубрики']] = 0
    pdf_new_str = pdf_new_str.sort_values(['Рейтинг рубрики', 'Рейтинг по запросам, итого'])
    #pdf_new.to_excel('df_tx_grouped_normalize_final_rated.xlsx')

    dt_now = str(datetime.datetime.now())[2:-9].replace('-','').replace(':','').replace(' ','_')
    excel_file = f'files/requests_text_{name}_{dt_now}.xlsx'
    print(excel_file)

    pdf_new_str.to_excel(excel_file, index=False)

In [15]:
color_list_full = ['#F96A6C', '#F96B6C', '#F96C6C', '#F96D6C', '#F96E6C', '#F96F6D', '#F9706D', '#F9716D', '#F9726D', '#F9736D', '#F9746E', '#F9756E', '#F9766E', '#F9776E', '#F9786E', '#F9796E', '#F9796F', '#F97A6F'
                       , '#F97B6F', '#F97C6F', '#FA7C6F', '#FA7D6F', '#FA7E6F', '#FA7E70', '#FA7F70', '#FA8070', '#FA8170', '#FA8270', '#FA8370', '#FA8471', '#FA8571', '#FA8671', '#FA8771', '#FA8871', '#FA8972', '#FA8A72'
                       , '#FA8B72', '#FA8C72', '#FA8D72', '#FA8E73', '#FB8F73', '#FB9073', '#FB9173', '#FB9273', '#FB9373', '#FB9374', '#FB9474', '#FB9574', '#FB9674', '#FB9774', '#FB9874', '#FB9875', '#FB9975', '#FB9A75'
                       , '#FB9B75', '#FB9C75', '#FB9D75', '#FB9E76', '#FB9F76', '#FBA076', '#FBA176', '#FCA176', '#FCA276', '#FCA377', '#FCA477', '#FCA577', '#FCA677', '#FCA777', '#FCA877', '#FCA878', '#FCA978', '#FCAA78'
                       , '#FCAB78', '#FCAC78', '#FCAD78', '#FCAD79', '#FCAE79', '#FCAF79', '#FCB079', '#FCB179', '#FCB279', '#FCB27A', '#FCB37A', '#FDB47A', '#FDB57A', '#FDB67A', '#FDB77A', '#FDB87B', '#FDB97B', '#FDBA7B'
                       , '#FDBB7B', '#FDBC7B', '#FDBD7C', '#FDBE7C', '#FDBF7C', '#FDC07C', '#FDC17C', '#FDC27C', '#FDC27D', '#FDC37D', '#FDC47D', '#FDC57D', '#FDC67D', '#FEC77D', '#FEC77E', '#FEC87E', '#FEC97E', '#FECA7E'
                       , '#FECB7E', '#FECC7E', '#FECC7F', '#FECD7F', '#FECE7F', '#FECF7F', '#FED07F', '#FED17F', '#FED280', '#FED380', '#FED480', '#FED580', '#FED680', '#FED781', '#FED881', '#FFD981', '#FFDA81', '#FFDB81'
                       , '#FFDC81', '#FFDC82', '#FFDD82', '#FFDE82', '#FFDF82', '#FFE082', '#FFE182', '#FFE183', '#FFE283', '#FFE383', '#FFE483', '#FFE583', '#FFE683', '#FFE784', '#FFE884', '#FFE984', '#FFEA84', '#FFEB84'
                       , '#FEEA83', '#FDEA83', '#FCEA83', '#FBEA83', '#FAE983', '#F9E983', '#F8E983', '#F7E883', '#F6E883', '#F5E883', '#F4E883', '#F4E783', '#F3E783', '#F2E783', '#F1E783', '#F0E683', '#EFE683', '#EEE683'
                       ,'#EDE683', '#EDE582', '#ECE582', '#EBE582', '#EAE582', '#EAE482', '#E9E482', '#E8E482', '#E7E482', '#E6E482', '#E6E382', '#E5E382', '#E4E382', '#E3E382', '#E3E282', '#E2E282', '#E1E282', '#E0E282'
                       , '#DFE182', '#DEE182', '#DDE182', '#DCE182', '#DCE081', '#DBE081', '#DAE081', '#D9E081', '#D8DF81', '#D7DF81', '#D6DF81', '#D5DF81', '#D4DE81', '#D3DE81', '#D2DE81', '#D1DD81', '#D0DD81', '#CFDD81'
                       , '#CEDD81', '#CDDC81', '#CCDC81', '#CBDC81', '#CADB80', '#C9DB80', '#C8DB80', '#C7DB80', '#C7DA80', '#C6DA80', '#C5DA80', '#C4DA80', '#C3D980', '#C2D980', '#C1D980', '#C0D980', '#C0D880', '#BFD880'
                       , '#BED880', '#BDD880', '#BDD780', '#BCD780', '#BBD780', '#BAD780', '#B9D780', '#B9D67F', '#B8D67F', '#B7D67F', '#B6D67F', '#B6D57F', '#B5D57F', '#B4D57F', '#B3D57F', '#B2D47F', '#B1D47F', '#B0D47F'
                       , '#AFD47F', '#AED37F', '#ADD37F', '#ACD37F', '#ABD27F', '#AAD27F', '#A9D27F', '#A8D27F', '#A7D17E', '#A6D17E', '#A5D17E', '#A4D07E', '#A3D07E', '#A2D07E', '#A1D07E', '#A0CF7E', '#9FCF7E', '#9ECF7E'
                       , '#9DCE7E', '#9CCE7E', '#9BCE7E', '#9ACE7E', '#9ACD7E', '#99CD7E', '#98CD7E', '#97CD7E', '#96CC7D', '#95CC7D', '#94CC7D', '#93CC7D', '#93CB7D', '#92CB7D', '#91CB7D', '#90CB7D', '#90CA7D', '#8FCA7D'
                       , '#8ECA7D', '#8DCA7D', '#8CCA7D', '#8CC97D', '#8BC97D', '#8AC97D', '#89C97D', '#88C87D', '#87C87D', '#86C87D', '#85C77C', '#84C77C', '#83C77C', '#82C77C', '#81C67C', '#80C67C', '#7FC67C', '#7EC57C'
                       , '#7DC57C', '#7CC57C', '#7BC57C', '#7AC47C', '#79C47C', '#78C47C', '#77C37C', '#76C37C', '#75C37C', '#74C37C', '#73C27B'
                       , '#72C27B', '#71C27B', '#70C17B', '#6FC17B', '#6EC17B', '#6DC17B', '#6DC07B', '#6CC07B', '#6BC07B', '#6AC07B', '#69BF7B', '#68BF7B', '#67BF7B', '#66BF7B', '#66BE7B', '#65BE7B', '#64BE7B', '#63BE7B']

def make_sep(number_str):
    return '{0:,}'.format(number_str).replace(',', ' ')

def render_mpl_table(data, file_name, index_col_to_colorize=5, dups=None, col_width=1.0, row_height=0.625, font_size=18,
                     header_color='#C0504D', row_colors=['#F2DCDB', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array([17.5, 10.8]))
        fig, ax = plt.subplots(figsize=size, dpi=2**8)
        ax.axis('off')

    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, colLoc='center', loc='bottom')

    mpl_table.auto_set_font_size(False)
    cells_dict = dict(six.iteritems(mpl_table._cells))
    
    im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\Лого_2ГИС_230401_1600.png")
    ax.figure.figimage(im, 58, 44, zorder=1, alpha=0.5)

    im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\лого_ШТ_230401_1600.png")
    ax.figure.figimage(im, 58, 1268, zorder=1, alpha=0.5)
    
    fpath = Path(mpl.get_data_path(), r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\arialnarrow.ttf")
    #from colour import Color
    #colors = [i.get_hex() for i in list(Color("red").range_to(Color("green"),data.iloc[:20].shape[0]))]
    color_list_full = ['#F96A6C', '#F96B6C', '#F96C6C', '#F96D6C', '#F96E6C', '#F96F6D', '#F9706D', '#F9716D', '#F9726D', '#F9736D', '#F9746E', '#F9756E', '#F9766E', '#F9776E', '#F9786E', '#F9796E', '#F9796F', '#F97A6F'
                       , '#F97B6F', '#F97C6F', '#FA7C6F', '#FA7D6F', '#FA7E6F', '#FA7E70', '#FA7F70', '#FA8070', '#FA8170', '#FA8270', '#FA8370', '#FA8471', '#FA8571', '#FA8671', '#FA8771', '#FA8871', '#FA8972', '#FA8A72'
                       , '#FA8B72', '#FA8C72', '#FA8D72', '#FA8E73', '#FB8F73', '#FB9073', '#FB9173', '#FB9273', '#FB9373', '#FB9374', '#FB9474', '#FB9574', '#FB9674', '#FB9774', '#FB9874', '#FB9875', '#FB9975', '#FB9A75'
                       , '#FB9B75', '#FB9C75', '#FB9D75', '#FB9E76', '#FB9F76', '#FBA076', '#FBA176', '#FCA176', '#FCA276', '#FCA377', '#FCA477', '#FCA577', '#FCA677', '#FCA777', '#FCA877', '#FCA878', '#FCA978', '#FCAA78'
                       , '#FCAB78', '#FCAC78', '#FCAD78', '#FCAD79', '#FCAE79', '#FCAF79', '#FCB079', '#FCB179', '#FCB279', '#FCB27A', '#FCB37A', '#FDB47A', '#FDB57A', '#FDB67A', '#FDB77A', '#FDB87B', '#FDB97B', '#FDBA7B'
                       , '#FDBB7B', '#FDBC7B', '#FDBD7C', '#FDBE7C', '#FDBF7C', '#FDC07C', '#FDC17C', '#FDC27C', '#FDC27D', '#FDC37D', '#FDC47D', '#FDC57D', '#FDC67D', '#FEC77D', '#FEC77E', '#FEC87E', '#FEC97E', '#FECA7E'
                       , '#FECB7E', '#FECC7E', '#FECC7F', '#FECD7F', '#FECE7F', '#FECF7F', '#FED07F', '#FED17F', '#FED280', '#FED380', '#FED480', '#FED580', '#FED680', '#FED781', '#FED881', '#FFD981', '#FFDA81', '#FFDB81'
                       , '#FFDC81', '#FFDC82', '#FFDD82', '#FFDE82', '#FFDF82', '#FFE082', '#FFE182', '#FFE183', '#FFE283', '#FFE383', '#FFE483', '#FFE583', '#FFE683', '#FFE784', '#FFE884', '#FFE984', '#FFEA84', '#FFEB84'
                       , '#FEEA83', '#FDEA83', '#FCEA83', '#FBEA83', '#FAE983', '#F9E983', '#F8E983', '#F7E883', '#F6E883', '#F5E883', '#F4E883', '#F4E783', '#F3E783', '#F2E783', '#F1E783', '#F0E683', '#EFE683', '#EEE683'
                       ,'#EDE683', '#EDE582', '#ECE582', '#EBE582', '#EAE582', '#EAE482', '#E9E482', '#E8E482', '#E7E482', '#E6E482', '#E6E382', '#E5E382', '#E4E382', '#E3E382', '#E3E282', '#E2E282', '#E1E282', '#E0E282'
                       , '#DFE182', '#DEE182', '#DDE182', '#DCE182', '#DCE081', '#DBE081', '#DAE081', '#D9E081', '#D8DF81', '#D7DF81', '#D6DF81', '#D5DF81', '#D4DE81', '#D3DE81', '#D2DE81', '#D1DD81', '#D0DD81', '#CFDD81'
                       , '#CEDD81', '#CDDC81', '#CCDC81', '#CBDC81', '#CADB80', '#C9DB80', '#C8DB80', '#C7DB80', '#C7DA80', '#C6DA80', '#C5DA80', '#C4DA80', '#C3D980', '#C2D980', '#C1D980', '#C0D980', '#C0D880', '#BFD880'
                       , '#BED880', '#BDD880', '#BDD780', '#BCD780', '#BBD780', '#BAD780', '#B9D780', '#B9D67F', '#B8D67F', '#B7D67F', '#B6D67F', '#B6D57F', '#B5D57F', '#B4D57F', '#B3D57F', '#B2D47F', '#B1D47F', '#B0D47F'
                       , '#AFD47F', '#AED37F', '#ADD37F', '#ACD37F', '#ABD27F', '#AAD27F', '#A9D27F', '#A8D27F', '#A7D17E', '#A6D17E', '#A5D17E', '#A4D07E', '#A3D07E', '#A2D07E', '#A1D07E', '#A0CF7E', '#9FCF7E', '#9ECF7E'
                       , '#9DCE7E', '#9CCE7E', '#9BCE7E', '#9ACE7E', '#9ACD7E', '#99CD7E', '#98CD7E', '#97CD7E', '#96CC7D', '#95CC7D', '#94CC7D', '#93CC7D', '#93CB7D', '#92CB7D', '#91CB7D', '#90CB7D', '#90CA7D', '#8FCA7D'
                       , '#8ECA7D', '#8DCA7D', '#8CCA7D', '#8CC97D', '#8BC97D', '#8AC97D', '#89C97D', '#88C87D', '#87C87D', '#86C87D', '#85C77C', '#84C77C', '#83C77C', '#82C77C', '#81C67C', '#80C67C', '#7FC67C', '#7EC57C'
                       , '#7DC57C', '#7CC57C', '#7BC57C', '#7AC47C', '#79C47C', '#78C47C', '#77C37C', '#76C37C', '#75C37C', '#74C37C', '#73C27B'
                       , '#72C27B', '#71C27B', '#70C17B', '#6FC17B', '#6EC17B', '#6DC17B', '#6DC07B', '#6CC07B', '#6BC07B', '#6AC07B', '#69BF7B', '#68BF7B', '#67BF7B', '#66BF7B', '#66BE7B', '#65BE7B', '#64BE7B', '#63BE7B']
    colors = ([color_list_full[i] for i in range(0, len(color_list_full), len(color_list_full) % 20)] + [color_list_full[-1]])
    
    for k in sorted(cells_dict.keys()): #k[0] - строка, k[1] - столбец
        cell = cells_dict[k]
        cell.set_edgecolor(edge_color)
        row_index = k[0]
        column_index = k[1]
        
        # Высота и цвет заголовков
        if row_index == 0:
            rotation='horizontal' if k == (0,0) else 'vertical' 
            cell.set_text_props(color='w', rotation=rotation, ha='center')
            cell.set_facecolor(header_color)
            cell.set_height(0.07)
            
        # Высота и цвет остальных строк
        else:
            ha = 'right' if k[1] !=0 else 'left'
            cell.set_facecolor(row_colors[k[0]%len(row_colors)])
            cell.set_height(0.008)
            cell.set_text_props(wrap=True, ha=ha)
        
        # Ширина столбцов
        if column_index == 0:
            cell.set_width(0.006)
        else:
            cell.set_width(0.0011)
        
        # Раскраска дублей
        if dups is not None:
            if row_index > 0 and column_index == 0 and cell.get_text().get_text() in dups:
                cell.set_facecolor('#FFC7CE')
                cell.set_text_props(color='#9C0006')
                
        # Цветвоая шкала
        if row_index > 0 and column_index == index_col_to_colorize:
            cell.set_facecolor(colors.pop(0).upper()) 
        
        # Шрифт
        cell.set_text_props(font=fpath, fontsize=font_size)
    fig.savefig(f'{file_name}', bbox_inches='tight')
    plt.close()
    
def add_image(docx_name, image_name, word='', truncate=False, with_images=True, img_w=17.5, img_h=10.8, blank=False, word_sample='files/report.docx'):
    if not word:
        word = 'Восторг и презрение масс захватили рюмочные, купальники, аренда коттеджей и гостевых домов, быстрое питание, паркинги, точки продаж безалкогольных напитков и стендап-клубы. Сюда же попали кафе-кондитерские, рестораны и системы нагревания табака glo:'
    if truncate:
        doc = docx.Document(word_sample)
        doc.save(docx_name)
    try:
        doc = docx.Document(docx_name)
    except:
        doc = docx.Document(word_sample)
        doc.save(docx_name)
    if blank:
        word=''
    doc.add_paragraph(word)
    if with_images:
        image_to_add = doc.add_picture(image_name)
        image_to_add.width = Cm(img_w)
        image_to_add.height = Cm(img_h)
    doc.save(docx_name)

        
def copy_images_texts(excel_images_from, text_from, dates=False, start_param=8):
    doc = docx.Document(text_from)
    current_text = [i.text for i in doc.paragraphs if i.text][start_param:]

    input_filename = excel_images_from #'files/moscow_requests.xlsx'
    dt_now = str(datetime.datetime.now())[2:-9].replace('-','').replace(':','').replace(' ','_')
    word_file = f'{input_filename[:-5]}_report_{dt_now}.docx'
    img_file_name = input_filename.replace('files/','')[:-5]
    
    print(word_file)
    j = 0
    i = 1
    for t in tqdm(current_text):
        if dates:
            if j==4:
                j = 0
                i+=1
            j+=1
            img_file_name_final = f'pictures/{img_file_name}_{i:02}_{j}.png'
        else:
            img_file_name_final = f'pictures/{img_file_name}_{i:02}.png'
            i+=1
        add_image(word_file, img_file_name_final, t)
    return current_text


def add_image_request(docx_name, image_name, img_w=17.5, img_h=10.8, word_sample='files/report.docx'):
    try:
        doc = docx.Document(docx_name)
    except:
        doc = docx.Document(word_sample)
        doc.save(docx_name)
    image_to_add = doc.add_picture(image_name)
    image_to_add.width = Cm(img_w)
    image_to_add.height = Cm(img_h)
    doc.save(docx_name)

    
def add_links(docx_name, rubric, words, city):
    try:
        doc = docx.Document(docx_name)
    except:
        doc = docx.Document('files/report.docx')
        doc.save(docx_name)
    p = doc.add_paragraph(rubric + ': ')
    for word in words:
        text = word
        query = word.replace(' ', '+')
        search = f'https://www.google.ru/search?q={query}+{city}'
        add_hyperlink(p, text, search)
        #doc.add_paragraph(', ')
        if word != words[-1]:
            p.add_run(', ')
    doc.save(docx_name)

def add_hyperlink(paragraph, text, url):
    # This gets access to the document.xml.rels file and gets a new relation id value
    part = paragraph.part
    r_id = part.relate_to(url, docx.opc.constants.RELATIONSHIP_TYPE.HYPERLINK, is_external=True)

    # Create the w:hyperlink tag and add needed values
    hyperlink = docx.oxml.shared.OxmlElement('w:hyperlink')
    hyperlink.set(docx.oxml.shared.qn('r:id'), r_id, )

    # Create a w:r element and a new w:rPr element
    new_run = docx.oxml.shared.OxmlElement('w:r')
    rPr = docx.oxml.shared.OxmlElement('w:rPr')

    # Join all the xml elements together add add the required text to the w:r element
    new_run.append(rPr)
    new_run.text = text
    hyperlink.append(new_run)

    # Create a new Run object and add the hyperlink into it
    r = paragraph.add_run()
    r._r.append (hyperlink)

    # A workaround for the lack of a hyperlink style (doesn't go purple after using the link)
    # Delete this if using a template that has the hyperlink style in it
    #r.font.color.theme_color = MSO_THEME_COLOR_INDEX.HYPERLINK
    #r.font.underline = True

    return hyperlink

def make_images(file_name, city, header_color, row_color, cnt=9999, size_x=17.5, size_y=10.8, font_size=18, img_w=17.5, img_h=10.8, blank=False, word_sample='files/report.docx'):
    dt_now = str(datetime.datetime.now())[2:-9].replace('-','').replace(':','').replace(' ','_')
    df = pd.read_excel(file_name).drop(['tx_1','tx_2','tx_3','tx_4','tx_5'], axis=1)
    word_file = f'{file_name[:-5]}_report_{dt_now}.docx'
    print(word_file)
    
    df.columns = ['Рубрика', 'Поисковый запрос', 'Количество\nзапросов 2020',
           'Количество\nзапросов 2021', 'Количество\nзапросов 2022',
           'Количество\nзапросов 2023', 'Количество\nзапросов, итого',
           'Рейтинг\nпо запросам, итого', 'Рейтинг рубрики']

    for i in df.columns[2:-2]:
        df[i] = df[i].apply(make_sep)
        
    dups = df['Поисковый запрос'][(df['Поисковый запрос'].duplicated())&(df['Рейтинг рубрики'] > 0)].values 
    df_text = pd.read_excel(file_name, usecols=['Рубрика', 'Рейтинг рубрики', 'Поисковый запрос', 'tx_1'])
    df_text = df_text[df_text['tx_1'].notna()]
    tx_1 = df_text[df_text['Поисковый запрос'].apply(lambda x: x not in dups)][['tx_1', 'Рейтинг рубрики']]
    
    for i in tqdm(df['Рейтинг рубрики'].unique()[1:cnt+1]):  
        dfx = df[df['Рейтинг рубрики'] == i].drop('Рубрика', axis=1).iloc[:20]
        rubric = df[df['Рейтинг рубрики'] == i]['Рубрика'].iloc[0]
        col_name = f'Поисковые запросы по рубрике\n{i:02}. {rubric}'
        img_file_name = file_name.replace('files/','')[:-5]
        img_file_name_final = f'pictures/{img_file_name}_{i:02}.png'
        dfx = dfx.rename({'Поисковый запрос':col_name}, axis=1).drop(['Рейтинг рубрики', 'Рейтинг\nпо запросам, итого'], axis=1)
        words = tx_1[tx_1['Рейтинг рубрики'] == i]['tx_1'].values
        
        render_mpl_table(dfx, img_file_name_final, font_size=19, header_color=header_color, row_colors=[row_color, 'w'], dups=dups)
        if blank:
            try:
                doc = docx.Document(word_file)
            except:
                doc = docx.Document('files/report_ten_city.docx')
                doc.save(word_file)
            word=''
            doc.add_paragraph(word)
            doc.save(word_file)
        else:
            add_links(word_file, rubric, words, city)
        add_image_request(word_file, img_file_name_final, img_w=img_w, img_h=img_h, word_sample=word_sample)
        
make_requests_word = make_images

In [10]:
def colorize_column(df, cols_not_to_color=[0], filter_zero=False):
    colors_dict = {}
    color_list_full = ['#F96A6C', '#F96B6C', '#F96C6C', '#F96D6C', '#F96E6C', '#F96F6D', '#F9706D', '#F9716D', '#F9726D', '#F9736D', '#F9746E', '#F9756E', '#F9766E', '#F9776E', '#F9786E', '#F9796E', '#F9796F', '#F97A6F'
                   , '#F97B6F', '#F97C6F', '#FA7C6F', '#FA7D6F', '#FA7E6F', '#FA7E70', '#FA7F70', '#FA8070', '#FA8170', '#FA8270', '#FA8370', '#FA8471', '#FA8571', '#FA8671', '#FA8771', '#FA8871', '#FA8972', '#FA8A72'
                   , '#FA8B72', '#FA8C72', '#FA8D72', '#FA8E73', '#FB8F73', '#FB9073', '#FB9173', '#FB9273', '#FB9373', '#FB9374', '#FB9474', '#FB9574', '#FB9674', '#FB9774', '#FB9874', '#FB9875', '#FB9975', '#FB9A75'
                   , '#FB9B75', '#FB9C75', '#FB9D75', '#FB9E76', '#FB9F76', '#FBA076', '#FBA176', '#FCA176', '#FCA276', '#FCA377', '#FCA477', '#FCA577', '#FCA677', '#FCA777', '#FCA877', '#FCA878', '#FCA978', '#FCAA78'
                   , '#FCAB78', '#FCAC78', '#FCAD78', '#FCAD79', '#FCAE79', '#FCAF79', '#FCB079', '#FCB179', '#FCB279', '#FCB27A', '#FCB37A', '#FDB47A', '#FDB57A', '#FDB67A', '#FDB77A', '#FDB87B', '#FDB97B', '#FDBA7B'
                   , '#FDBB7B', '#FDBC7B', '#FDBD7C', '#FDBE7C', '#FDBF7C', '#FDC07C', '#FDC17C', '#FDC27C', '#FDC27D', '#FDC37D', '#FDC47D', '#FDC57D', '#FDC67D', '#FEC77D', '#FEC77E', '#FEC87E', '#FEC97E', '#FECA7E'
                   , '#FECB7E', '#FECC7E', '#FECC7F', '#FECD7F', '#FECE7F', '#FECF7F', '#FED07F', '#FED17F', '#FED280', '#FED380', '#FED480', '#FED580', '#FED680', '#FED781', '#FED881', '#FFD981', '#FFDA81', '#FFDB81'
                   , '#FFDC81', '#FFDC82', '#FFDD82', '#FFDE82', '#FFDF82', '#FFE082', '#FFE182', '#FFE183', '#FFE283', '#FFE383', '#FFE483', '#FFE583', '#FFE683', '#FFE784', '#FFE884', '#FFE984', '#FFEA84', '#FFEB84'
                   , '#FEEA83', '#FDEA83', '#FCEA83', '#FBEA83', '#FAE983', '#F9E983', '#F8E983', '#F7E883', '#F6E883', '#F5E883', '#F4E883', '#F4E783', '#F3E783', '#F2E783', '#F1E783', '#F0E683', '#EFE683', '#EEE683'
                   ,'#EDE683', '#EDE582', '#ECE582', '#EBE582', '#EAE582', '#EAE482', '#E9E482', '#E8E482', '#E7E482', '#E6E482', '#E6E382', '#E5E382', '#E4E382', '#E3E382', '#E3E282', '#E2E282', '#E1E282', '#E0E282'
                   , '#DFE182', '#DEE182', '#DDE182', '#DCE182', '#DCE081', '#DBE081', '#DAE081', '#D9E081', '#D8DF81', '#D7DF81', '#D6DF81', '#D5DF81', '#D4DE81', '#D3DE81', '#D2DE81', '#D1DD81', '#D0DD81', '#CFDD81'
                   , '#CEDD81', '#CDDC81', '#CCDC81', '#CBDC81', '#CADB80', '#C9DB80', '#C8DB80', '#C7DB80', '#C7DA80', '#C6DA80', '#C5DA80', '#C4DA80', '#C3D980', '#C2D980', '#C1D980', '#C0D980', '#C0D880', '#BFD880'
                   , '#BED880', '#BDD880', '#BDD780', '#BCD780', '#BBD780', '#BAD780', '#B9D780', '#B9D67F', '#B8D67F', '#B7D67F', '#B6D67F', '#B6D57F', '#B5D57F', '#B4D57F', '#B3D57F', '#B2D47F', '#B1D47F', '#B0D47F'
                   , '#AFD47F', '#AED37F', '#ADD37F', '#ACD37F', '#ABD27F', '#AAD27F', '#A9D27F', '#A8D27F', '#A7D17E', '#A6D17E', '#A5D17E', '#A4D07E', '#A3D07E', '#A2D07E', '#A1D07E', '#A0CF7E', '#9FCF7E', '#9ECF7E'
                   , '#9DCE7E', '#9CCE7E', '#9BCE7E', '#9ACE7E', '#9ACD7E', '#99CD7E', '#98CD7E', '#97CD7E', '#96CC7D', '#95CC7D', '#94CC7D', '#93CC7D', '#93CB7D', '#92CB7D', '#91CB7D', '#90CB7D', '#90CA7D', '#8FCA7D'
                   , '#8ECA7D', '#8DCA7D', '#8CCA7D', '#8CC97D', '#8BC97D', '#8AC97D', '#89C97D', '#88C87D', '#87C87D', '#86C87D', '#85C77C', '#84C77C', '#83C77C', '#82C77C', '#81C67C', '#80C67C', '#7FC67C', '#7EC57C'
                   , '#7DC57C', '#7CC57C', '#7BC57C', '#7AC47C', '#79C47C', '#78C47C', '#77C37C', '#76C37C', '#75C37C', '#74C37C', '#73C27B'
                   , '#72C27B', '#71C27B', '#70C17B', '#6FC17B', '#6EC17B', '#6DC17B', '#6DC07B', '#6CC07B', '#6BC07B', '#6AC07B', '#69BF7B', '#68BF7B', '#67BF7B', '#66BF7B', '#66BE7B', '#65BE7B', '#64BE7B', '#63BE7B']
    cols_to_color = [i for i in range(len(df.columns)) if i not in cols_not_to_color]
    
    for column_index in cols_to_color:
        column_name = df.columns[column_index]
        if filter_zero:
            df = df[df[column_name]>0]
        df = df.sort_values(column_name, ascending=False)
        ind = df[[column_name]].index
        colors_for_col = []
        num = len(ind) // len(color_list_full)
        diff = len(ind) % len(color_list_full)
        step = len(color_list_full) // diff
        extra_list = list(range(0, len(color_list_full), step))

        for i1 in range(len(color_list_full)):
            color = color_list_full[i1]
            if len(color_list_full) < len(ind):
                for _ in range(num):
                    if len(colors_for_col) < len(ind):
                        colors_for_col.append(color)
                if i1 in extra_list and len(colors_for_col) < len(ind):
                    colors_for_col.append(color)
            else:
                for i in range(0, len(color_list_full), len(color_list_full) // len(ind)):
                    colors_for_col.append(color_list_full[i])
                colors_for_col.append([color_list_full[-1]])
        column_index = list(df.columns).index(column_name)
        colors_df = pd.DataFrame(zip(ind, colors_for_col), columns=[0, column_index]).set_index(0)
        df_merged = df.merge(colors_df, left_index=True, right_index=True)
        new_colors_dict = df_merged.iloc[:,-1].to_dict()
        colors_dict.update({column_index:new_colors_dict})
    return colors_dict

def make_ratings_image(df, output_file_name=None, cols_without_color=[0], dedup_cols=None, col_width=1.0, row_height=0.625, font_size=18,
                     header_color='#C0504D', row_colors=['#F2DCDB', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, size_x=17.5, size_y=10.8, **kwargs):
    rating_col = df.iloc[:,-1].name
    data = df[df[rating_col]>0].sort_values(rating_col)
    data_separeted_numbers = data.copy()
    for col in data.columns[1:]:
        data_separeted_numbers[col] = data[col].apply(make_sep)
        if col in ['Пользователи, %', 'Запросы, %', 'Запросов на пользователя,\nитого']:
            data_separeted_numbers[col] = data[col].apply(lambda x: str(x).replace('.',','))
        
    if ax is None:
        size = (np.array([size_x, size_y]))
        fig, ax = plt.subplots(figsize=size, dpi=2**8)
        ax.axis('off')

    mpl_table = ax.table(cellText=data_separeted_numbers.values, bbox=bbox, colLabels=data.columns, colLoc='center', loc='bottom')
    mpl_table.auto_set_font_size(False)
    cells_dict = dict(six.iteritems(mpl_table._cells))
    
    im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\Лого_2ГИС_230401_1600.png")
    ax.figure.figimage(im, 43, 44, zorder=1, alpha=0.5)

    im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\лого_ШТ_230401_1600.png")
    ax.figure.figimage(im, 43, 1068, zorder=1, alpha=0.5)
    
    fpath = Path(mpl.get_data_path(), r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\arialnarrow.ttf")
    colors_dict = colorize_column(df, cols_not_to_color=cols_without_color)
    colors_dict.update(colorize_column(data, cols_not_to_color=list(range(data.shape[1]-1))), filter_zero=True)
    
    for k in sorted(cells_dict.keys()): #k[0] - строка, k[1] - столбец
        cell = cells_dict[k]
        cell.set_edgecolor(edge_color)
        row_index = k[0]
        column_index = k[1]
        
        # Высота и цвет заголовков
        if row_index == 0:
            rotation='horizontal' if k == (0,0) else 'vertical' 
            cell.set_text_props(color='w', rotation=rotation, ha='center')
            cell.set_facecolor(header_color)
            cell.set_height(0.08)
            
        # Высота и цвет остальных строк
        else:
            ha = 'left' if column_index ==0 else 'right'
            cell.set_facecolor(row_colors[k[0]%len(row_colors)])
            cell.set_height(0.008)
            cell.set_text_props(wrap=True, ha=ha)
        
        # Ширина столбцов
        if column_index == 0:
            cell.set_width(0.006)
        else:
            cell.set_width(0.0012)
                        
        # Цветвоая шкала
        if row_index > 0 and column_index not in cols_without_color:
            row_index_origin = data[data.iloc[:,-1] == row_index].index[0]
            color_value = colors_dict[column_index][row_index_origin]
            cell.set_facecolor(color_value) 
        
        # Раскраска дублей
        if dedup_cols is not None:
            for i in dedup_cols:
                dups = df.iloc[:,i][(df.iloc[:,i].duplicated())&(('Рейтинг рубрики' not in df.columns)|(df['Рейтинг рубрики'] > 0))].values 
                if row_index > 0 and column_index == i and cell.get_text().get_text() in dups:
                    cell.set_facecolor('#FFC7CE')
                    cell.set_text_props(color='#9C0006')         
        # Шрифт
        cell.set_text_props(font=fpath, fontsize=font_size)
    if output_file_name:
        fig.savefig(f'{output_file_name}', bbox_inches='tight')
    plt.close()    

def make_ratings_word(file_name, header_color, row_color, cnt=9999, size_x=17.5, size_y=10.8, font_size=18, img_w=17.5, img_h=10.8, blank=False, word_sample='files/report.docx'):
    dt_now = str(datetime.datetime.now())[2:-9].replace('-','').replace(':','').replace(' ','_')
    df = pd.read_excel(file_name)
    word_file = f'{file_name[:-17]}_report_{dt_now}.docx'
    print(word_file)
    
    df = df[(df['№']!='Итог')]
    cols = [
        'Рубрика' 
        ,'Пользователи, итого'
        ,'Пользователи, %'
        ,'Запросы, итого'
        ,'Запросы, %'
        ,'Запросов на пользователя,\nитого'
    ]

    ratings_cols = [i for i in df.columns if 'ейтинг' in i and i not in  ('В рейтингах, раз','Рейтинг суммарный','Рейтинг средний','Вне рейтингов')]

    for col in ratings_cols:
        df[col] = df[col].astype('int').round()

    for col in ['Пользователи, %', 'Запросы, %', 'Запросов на пользователя,\nитого']:
        df[col] = df[col].astype('float64').round(1)
    
    i=0
    for col in tqdm(ratings_cols[:cnt]):  
        i+=1
        cols_filter = cols+[col]    
        dfx = df[df[col]>0].sort_values(col)['Рубрика'].values
        img_file_name = file_name.replace('files/','')[:-5]
        img_file_name_final = f'pictures/{img_file_name}_{i:02}.png'
        make_ratings_image(df[cols_filter], output_file_name=img_file_name_final, cols_without_color=[0], header_color=header_color, row_colors=[row_color, 'w'], size_x=size_x, size_y=size_y, font_size=font_size)
        txt =  col + '\n' + ', '.join([word[::-1].replace('/', 'и', 1)[::-1].replace('/', ',').replace(' ,', ',').lower() for word in df[df[col]>0].sort_values(col)['Рубрика'].values]) + ":"
        add_image(word_file, img_file_name_final, word=txt, img_w=img_w, img_h=img_h, blank=blank, word_sample=word_sample)
   

In [ ]:
def count_dates(col, df_new='df_new'):
    dff1 = spark.sql(f'''

    with t as
        (
        select 
            year_r
            , rubricName
            , {col}
            , count(*) cnt
        from {df_new}
        group by
            year_r
            , rubricName
            , {col}
        order by
            year_r
            , rubricName
            , {col}
        )
    ,gr as
        (
        select
            rubricName 
            ,{col} 
            ,max(case when year_r = 2020 then cnt end) cnt_2020
            ,max(case when year_r = 2021 then cnt end) cnt_2021
            ,max(case when year_r = 2022 then cnt end) cnt_2022
            ,max(case when year_r = 2023 then cnt end) cnt_2023
        from t
        group by
            rubricName
            ,{col}
        order by
            rubricName
            ,{col}    
        )

    select
        rubricName 
        ,{col}
        ,sum(cnt_2020) over(partition by rubricName, {col}) / sum(cnt_2020) over(partition by rubricName)*100 cnt_2020
        ,sum(cnt_2021) over(partition by rubricName, {col}) / sum(cnt_2021) over(partition by rubricName)*100 cnt_2021
        ,sum(cnt_2022) over(partition by rubricName, {col}) / sum(cnt_2022) over(partition by rubricName)*100 cnt_2022
        ,sum(cnt_2023) over(partition by rubricName, {col}) / sum(cnt_2023) over(partition by rubricName)*100 cnt_2023
        
        ,case when row_number() over(partition by rubricName order by cnt_2020 desc) < 4 then row_number() over(partition by rubricName order by cnt_2020 desc) end max_2020
        ,case when row_number() over(partition by rubricName order by cnt_2021 desc) < 4 then row_number() over(partition by rubricName order by cnt_2021 desc) end max_2021
        ,case when row_number() over(partition by rubricName order by cnt_2022 desc) < 4 then row_number() over(partition by rubricName order by cnt_2022 desc) end max_2022
        ,case when row_number() over(partition by rubricName order by cnt_2023 desc) < 4 then row_number() over(partition by rubricName order by cnt_2023 desc) end max_2023
        
        ,case when row_number() over(partition by rubricName order by cnt_2020) < 4 then row_number() over(partition by rubricName order by cnt_2020) end min_2020
        ,case when row_number() over(partition by rubricName order by cnt_2021) < 4 then row_number() over(partition by rubricName order by cnt_2021) end min_2021
        ,case when row_number() over(partition by rubricName order by cnt_2022) < 4 then row_number() over(partition by rubricName order by cnt_2022) end min_2022
        ,case when row_number() over(partition by rubricName order by cnt_2023) < 4 then row_number() over(partition by rubricName order by cnt_2023) end min_2023
        
        ,case when row_number() over(partition by rubricName order by cnt_2021 - cnt_2020 desc) < 4 then row_number() over(partition by rubricName order by cnt_2021 - cnt_2020 desc) end up_2021
        ,case when row_number() over(partition by rubricName order by cnt_2022 - cnt_2021 desc) < 4 then row_number() over(partition by rubricName order by cnt_2022 - cnt_2021 desc) end up_2022
        ,case when row_number() over(partition by rubricName order by cnt_2023 - cnt_2022 desc) < 4 then row_number() over(partition by rubricName order by cnt_2023 - cnt_2022 desc) end up_2023
        
        ,case when row_number() over(partition by rubricName order by cnt_2021 - cnt_2020) < 4 then row_number() over(partition by rubricName order by cnt_2021 - cnt_2020) end down_2021
        ,case when row_number() over(partition by rubricName order by cnt_2022 - cnt_2021) < 4 then row_number() over(partition by rubricName order by cnt_2022 - cnt_2021) end down_2022
        ,case when row_number() over(partition by rubricName order by cnt_2023 - cnt_2022) < 4 then row_number() over(partition by rubricName order by cnt_2023 - cnt_2022) end down_2023
        
        ,  abs(max(cnt_2023) over(partition by rubricName) - max(cnt_2022) over(partition by rubricName))
         + abs(max(cnt_2022) over(partition by rubricName) - max(cnt_2021) over(partition by rubricName))
         + abs(max(cnt_2021) over(partition by rubricName) - max(cnt_2020) over(partition by rubricName))
         + abs(min(cnt_2023) over(partition by rubricName) - min(cnt_2022) over(partition by rubricName))
         + abs(min(cnt_2022) over(partition by rubricName) - min(cnt_2021) over(partition by rubricName))
         + abs(min(cnt_2021) over(partition by rubricName) - min(cnt_2020) over(partition by rubricName))
         delta_score
         
    from gr

    ''').toPandas()
    return dff1

In [ ]:
def make_dates_file(name):
    df = spark.read.parquet(f'files/df_filtered_{name}')
    df.createOrReplaceTempView('df')

    df_new = spark.sql('''

    select 
    rubricName
    , time
    , year(time) year_r
    , hour(time) hour_r
    , SUBSTR(time, 6, 5) date_r
    , case when weekofyear(time) > 9 then 1 else weekofyear(time) end week_r
    , case when extract(dayofweek from time) = 1 then 7
        else extract(dayofweek from time)-1
    end week_day_r

    from df
    ''')

    df_new.write.mode('overwrite').parquet(f'files/df_dates_{name}')

    df_new = spark.read.parquet(f'files/df_dates_{name}').where('year_r in (2020,2021,2022,2023) and month(time) < 3 ')
    df_new.createOrReplaceTempView('df_new')

    dff1 = count_dates('hour_r')
    dff2 = count_dates('date_r')
    dff3 = count_dates('week_r')
    dff4 = count_dates('week_day_r')

    t = ['Рубрика','Час','Количество запросов 2020','Количество запросов 2021','Количество запросов 2022','Количество запросов 2023',
        'max_2020','max_2021','max_2022','max_2023','min_2020','min_2021','min_2022','min_2023','up_2021','up_2022','up_2023','down_2021','down_2022','down_2023', 'delta_score']


    t[1] = 'Час'
    dff1.columns = t
    t[1] = 'Дата'
    dff2.columns = t
    t[1] = 'Неделя'
    dff3.columns = t
    t[1] = 'День недели'
    dff4.columns = t

    with pd.ExcelWriter(f'files/df_dates_{name}.xlsx') as writer:
        dff1.to_excel(writer, sheet_name='Часы', index=False)
        dff2.to_excel(writer, sheet_name='Даты', index=False)
        dff3.to_excel(writer, sheet_name='Недели', index=False)
        dff4.to_excel(writer, sheet_name='Дни недели', index=False)

In [16]:
def make_linear_plots(input_filename, rubrics_df, cnt=9999, size_x=17.5, size_y=10.8, font_size=16, img_w=17.5, img_h=10.8, blank=False, word_sample='files/report.docx'):
    def zero(x,y):
        if x < 0:
            return str(0) +'%'
        return str(round(x,1)) +'%'
    
    dt_now = str(datetime.datetime.now())[2:-9].replace('-','').replace(':','').replace(' ','_')
    word_file = f'{input_filename[:-5]}_report_{dt_now}.docx'
    print(word_file)
    cols1 = ['Рубрика','Час','Количество запросов 2020','Количество запросов 2021','Количество запросов 2022','Количество запросов 2023',]
    cols2 = ['Рубрика','Дата','Количество запросов 2020','Количество запросов 2021','Количество запросов 2022','Количество запросов 2023',]
    cols3 = ['Рубрика','Неделя','Количество запросов 2020','Количество запросов 2021','Количество запросов 2022','Количество запросов 2023',]
    cols4 = ['Рубрика','День недели','Количество запросов 2020','Количество запросов 2021','Количество запросов 2022','Количество запросов 2023',]
    dff1 = pd.read_excel(input_filename, sheet_name='Часы')[cols1]
    dff2 = pd.read_excel(input_filename, sheet_name='Даты')[cols2]
    dff3 = pd.read_excel(input_filename, sheet_name='Недели')[cols3]
    dff4 = pd.read_excel(input_filename, sheet_name='Дни недели')[cols4]
    
    i = 0
    for rubric in tqdm(rubrics_df[:].iloc[:,0].values[0:cnt]):
        dff1_test = dff1[dff1['Рубрика'] == rubric].set_index(dff1.columns[1])
        dff2_test = dff2[dff2['Рубрика'] == rubric].set_index(dff2.columns[1])
        dff3_test = dff3[dff3['Рубрика'] == rubric].set_index(dff3.columns[1])
        dff4_test = dff4[dff4['Рубрика'] == rubric].set_index(dff4.columns[1])
        dff4_test.index = (['пн','вт','ср','чт','пт','сб','вс'])
        dff1_test.index = dff1_test.index.astype(str)
        i+=1
        j=0
        for d in [dff1_test, dff2_test, dff3_test, dff4_test]: 
            j+=1
            fig, ax = plt.subplots()
            ax = sns.lineplot(data = d, linewidth=4, palette=['#4F81BD', '#9BBB59', '#F79646', '#C0504D'])
            plt.rcParams['figure.figsize']=size_x,size_y
            plt.rcParams['font.size'] = font_size
            plt.rcParams['figure.dpi'] = 2**8
            plt.rcParams['figure.facecolor'] = 'white'
            fpath = Path(mpl.get_data_path(), r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\arialnarrow.ttf")
            
            
            plt.legend(['2020', '2021', '2022', '2023'], prop=fpath)
            plt.xticks(font=fpath)
            plt.yticks(font=fpath)
            plt.grid(visible=True, linewidth=0.3)
            ax.set_xlabel('')
            ax.yaxis.set_major_formatter(FuncFormatter(zero))
            ax.xaxis.set_major_locator(ticker.MultipleLocator(1)) 
            ax.yaxis.set_major_locator(ticker.LinearLocator(numticks=6))  
            title_text = f'{i:02}/{j}. {rubric}. '
            img_file_name = input_filename.replace('files/','')[:-5]
            img_file_name_final = f'pictures/{img_file_name}_{i:02}_{j}.png'
            if 'Час' == d.index.name:
                ax.set_title(f'{title_text}Запросы по часам, шт.', font=fpath)
            elif 'Дата' == d.index.name:
                ax.xaxis.set_major_locator(ticker.MultipleLocator(4)) 
                ax.set_title(f'{title_text}Запросы по датам, шт.', font=fpath)
            elif 'Неделя' == d.index.name:
                ax.set_title(f'{title_text}Запросы по неделям, шт.', font=fpath)
            else:
                ax.set_title(f'{title_text}Запросы по дням недели, шт.', font=fpath) 
            
            im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\лого_2гис_232_354.png")
            ax.figure.figimage(im, 3250, 150, zorder=1, alpha=0.5)
          
            im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\лого_шт_232_354.png")
            ax.figure.figimage(im, 200, 1800, zorder=1, alpha=0.5)

            ax.figure.savefig(img_file_name_final, bbox_inches='tight')
            add_image(word_file, img_file_name_final, rubric, img_w=img_w, img_h=img_h, blank=blank,word_sample=word_sample)
            #plt.show()
            plt.close()

In [18]:
def plot_matrix(file, city, cnt_graphics=50, cnt_shars=10, step=1, size_x=6.89*4, size_y=4.25*4, font_size=16, img_w=17.5, img_h=10.8, blank=False, word_sample='files/report.docx'):
    shar_nums = list(range(1,16))*500
    start_range = 1
    stop_range = cnt_shars*step+1
    time = str(datetime.datetime.now())[2:16].replace('-','').replace(' ','_').replace(':','')
    print(f'files/{city}/{city}_shars_{time}.docx')
    df_list = []
    
    for graphic_num in tqdm(range(cnt_graphics)):
        shar_num = shar_nums.pop()

        df = pd.read_excel(file)[['Рубрика','Запросы 2022', 'Запросы 2023', 'Запросы, итого']].iloc[range(start_range, stop_range, step)].set_index('Рубрика').fillna(0)
        start_range = stop_range
        stop_range = (graphic_num+2)*cnt_shars*step+1

        df['Запросы, итого'] = (df['Запросы, итого'] - min(df['Запросы, итого'])) / (max(df['Запросы, итого']) - min(df['Запросы, итого']))*300
        requests_total = pd.read_excel(file)['Запросы 2023'].sum()
        df['Рост/Падение, %'] = df['Запросы 2023'] / df['Запросы 2022']
        df['Доля рынка, %'] = df['Запросы 2023'] / requests_total
        x = (df['Доля рынка, %'])
        y = (df['Рост/Падение, %'])
        sizes = (df['Запросы, итого']/1000)
        x_avg = (x.min()+x.max())/2
        y_avg = (y.min()+y.max())/2
        xi, yi = df['Доля рынка, %'], df['Рост/Падение, %']
        df.loc[(xi > x_avg) & (yi > y_avg), 'group'] = 'Звёзды:'
        df.loc[(xi > x_avg)  & (yi <= y_avg), 'group'] = 'Коровы:'
        df.loc[(xi <= x_avg)  & (yi > y_avg), 'group'] = 'Дети:'
        df.loc[(xi <= x_avg)  & (yi <= y_avg), 'group'] = 'Собаки:'
        df = df.sort_values(['Доля рынка, %', 'Рост/Падение, %'], ascending=[False, False])

        plt.rcParams['figure.figsize'] = size_x, size_y
        plt.rcParams['figure.dpi'] = 2**8
        plt.rcParams['font.size'] = '18'
        fpath = Path(mpl.get_data_path(), r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\arialnarrow.ttf")
        fig, ax = plt.subplots()
        ax.scatter(x, y, sizes, c='#C0504D', alpha=0.7)

        n = 0.02
        m = 0.02
        ax.set_ylim(ax.get_ylim()[0] * (1 - n), ax.get_ylim()[1] * (1 + n))
        ax.set_xlim(ax.get_xlim()[0] * (1 - m), ax.get_xlim()[1] * (1 + m))
        ax.spines['top'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_xticklabels('')
        ax.set_yticklabels('')
        plt.tick_params(bottom = False, left=False)

        for xi, yi, rubric, size in zip(x, y, x.index, sizes):
            rs = rubric.split(' ')
            if len(rs) > 1:
                rsl = len(rs) // 2
                rubric = ' '.join(rs[:rsl] + ['\n'] + rs[rsl:])

            shar_path = [r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\Шар_{:02}.png".format(shar_num)]
            shar_img = OffsetImage(plt.imread(shar_path[0], format="png"), zoom=size)
            ab = AnnotationBbox(shar_img, (xi, yi), frameon=False)
            ax.add_artist(ab)
            plt.annotate(rubric, (xi, yi), ha='center', va='center', font=fpath, color='#404040')

        ax.hlines((y.min()+y.max())/2, x.min(), x.max(), colors='gray', linewidth=1)
        ax.vlines((x.min()+x.max())/2, y.min(), y.max(), colors='gray', linewidth=1)
        
        im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\лого_шт_232_354_gray.png")
        newax = fig.add_axes([0.1, 0.8, 0.1, 0.1], zorder=1, alpha=0.5)
        newax.imshow(im)
        newax.axis('off')
        im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\лого_2гис_232_354_gray.png")
        newax = fig.add_axes([0.9, 0.1, 0.1, 0.1], zorder=1, alpha=0.5)
        newax.imshow(im)
        newax.axis('off')
        fig.savefig(f'pictures/{city}_shar_{shar_num}.png', bbox_inches='tight', transparent=False)
        
        txt = ''
        for cat in ['Звёзды:','Коровы:','Дети:','Собаки:']:
            txt +=  cat + ' ' + ', '.join([word[::-1].replace('/', 'и', 1)[::-1].replace('/', ',').replace(' ,', ',').lower() for word in df[df['group'] == cat].index]) + "\n"
        txt = txt[::-1].replace('\n',':', 1)[::-1]
        add_image(f'files/{city}/{city}_shars_{time}.docx', f'pictures/{city}_shar_{shar_num}.png', word=txt, img_w=img_w, img_h=img_h, blank=blank,word_sample=word_sample)
        #plt.show()
        plt.close()
        df_list.append(df)
    return pd.concat(df_list)

def plot_matrix_cities(cnt_graphics=100, show=False, size_x=6.89*4, size_y=4.25*4, font_size=16, img_w=17.5, img_h=10.8, blank=False, word_sample='files/report.docx'):
    folder='ten_city'
    shar_nums = list(range(1,16))*500
    time = str(datetime.datetime.now())[2:16].replace('-','').replace(' ','_').replace(':','')
    print(f'files/{folder}/{folder}_shars_{time}.docx')
    df_src = pd.read_parquet(f'files/ten_city/requests_for_shars')
    df_new = df_src[['rubricName','city']].drop_duplicates().set_index(['rubricName','city'])
    df_new['Запросы 2020'] = df_src[df_src['year_r'] == 2020].groupby(['rubricName', 'city']).sum('cnt_requests')[['cnt_requests']]
    df_new['Запросы 2021'] = df_src[df_src['year_r'] == 2021].groupby(['rubricName', 'city']).sum('cnt_requests')[['cnt_requests']]
    df_new['Запросы 2022'] = df_src[df_src['year_r'] == 2022].groupby(['rubricName', 'city']).sum('cnt_requests')[['cnt_requests']]
    df_new['Запросы 2023'] = df_src[df_src['year_r'] == 2023].groupby(['rubricName', 'city']).sum('cnt_requests')[['cnt_requests']]
    df_new = df_new.fillna(0).astype(int)
    df_new['Запросы, итого'] =  df_new['Запросы 2020'] + df_new['Запросы 2021'] + df_new['Запросы 2022'] + df_new['Запросы 2023']
    df_new = df_new.reset_index().set_index('city')

    rubrics = dd.read_parquet(f"files/ten_city/df_tx_grouped_filtered_ten_city")['rubricName'].unique().compute()
    rubrics = [i for i in rubrics.to_dict().values()]
    df_list = []
    
    for rubric in tqdm(rubrics[:cnt_graphics]):
        shar_num = shar_nums.pop()
        df = df_new.loc[df_new['rubricName']==rubric]
        df['Запросы, итого'] = (df['Запросы, итого'] - min(df['Запросы, итого'])) / (max(df['Запросы, итого']) - min(df['Запросы, итого']))*300
        requests_total = df_new['Запросы 2023'].sum()
        df['Рост/Падение, %'] = df['Запросы 2023'] / df['Запросы 2022']
        df['Доля рынка, %'] = df['Запросы 2023'] / requests_total
        x = (df['Доля рынка, %'])
        y = (df['Рост/Падение, %'])
        sizes = (df['Запросы, итого']/1000)
        x_avg = (x.min()+x.max())/2
        y_avg = (y.min()+y.max())/2
        xi, yi = df['Доля рынка, %'], df['Рост/Падение, %']
        df.loc[(xi > x_avg) & (yi > y_avg), 'group'] = 'Звёзды:'
        df.loc[(xi > x_avg)  & (yi <= y_avg), 'group'] = 'Коровы:'
        df.loc[(xi <= x_avg)  & (yi > y_avg), 'group'] = 'Дети:'
        df.loc[(xi <= x_avg)  & (yi <= y_avg), 'group'] = 'Собаки:'
        df = df.sort_values(['Доля рынка, %', 'Рост/Падение, %'], ascending=[False, False])

        plt.rcParams['figure.figsize'] = size_x, size_y
        plt.rcParams['figure.dpi'] = 2**8
        plt.rcParams['font.size'] = '18'
        fpath = Path(mpl.get_data_path(), r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\arialnarrow.ttf")
        fig, ax = plt.subplots()
        ax.scatter(x, y, sizes, c='#C0504D', alpha=0.7)

        n = 0.02
        m = 0.02
        ax.set_ylim(ax.get_ylim()[0] * (1 - n), ax.get_ylim()[1] * (1 + n))
        ax.set_xlim(ax.get_xlim()[0] * (1 - m), ax.get_xlim()[1] * (1 + m))
        ax.spines['top'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_xticklabels('')
        ax.set_yticklabels('')
        plt.tick_params(bottom = False, left=False)
        plt.title(rubric, font=fpath, fontsize=40, fontweight=1000)

        for xi, yi, city, size in zip(x, y, x.index, sizes):
            rs = city.split(' ')
            if len(rs) > 1:
                rsl = len(rs) // 2
                city = ' '.join(rs[:rsl] + ['\n'] + rs[rsl:])

            shar_path = [r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\Шар_{:02}.png".format(shar_num)]
            shar_img = OffsetImage(plt.imread(shar_path[0], format="png"), zoom=size)
            ab = AnnotationBbox(shar_img, (xi, yi), frameon=False)
            ax.add_artist(ab)
            plt.annotate(city, (xi, yi), ha='center', va='center', font=fpath, color='#404040')

        ax.hlines((y.min()+y.max())/2, x.min(), x.max(), colors='gray', linewidth=1)
        ax.vlines((x.min()+x.max())/2, y.min(), y.max(), colors='gray', linewidth=1)
        
        im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\лого_шт_232_354_gray.png")
        newax = fig.add_axes([0.1, 0.8, 0.1, 0.1], zorder=1, alpha=0.5)
        newax.imshow(im)
        newax.axis('off')
        im = plt.imread(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Лого\лого_2гис_232_354_gray.png")
        newax = fig.add_axes([0.9, 0.1, 0.1, 0.1], zorder=1, alpha=0.5)
        newax.imshow(im)
        newax.axis('off')
        fig.savefig(f'pictures/{folder}_shar_{shar_num}.png', bbox_inches='tight', transparent=False)
        
        txt = f'{rubric} '
        for cat in ['Звёзды:','Коровы:','Дети:','Собаки:']:
            txt +=  cat + ' ' + ', '.join([word[::-1].replace('/', 'и', 1)[::-1].replace('/', ',').replace(' ,', ',') for word in df[df['group'] == cat].index]) + "\n"
        txt = txt[::-1].replace('\n',':', 1)[::-1]
        add_image(f'files/{folder}/{folder}_shars_{time}.docx', f'pictures/{folder}_shar_{shar_num}.png', word=txt, img_w=img_w, img_h=img_h, blank=blank,word_sample=word_sample)
        if show:
            plt.show()
        plt.close()
        df_list.append(df)
    return pd.concat(df_list)

In [2]:
files_list = [['D:\\файлы_2гис_2023\\Калининград\\Калининград_2020_975659.csv',
  'D:\\файлы_2гис_2023\\Калининград\\Калининград_2021_975670.csv',
  'D:\\файлы_2гис_2023\\Калининград\\Калининград_2022_975672.csv',
  'D:\\файлы_2гис_2023\\Калининград\\Калининград_2023_975679.csv'],
 ['D:\\файлы_2гис_2023\\Астана\\Астана_2020_975007.csv',
  'D:\\файлы_2гис_2023\\Астана\\Астана_2021_975009.csv',
  'D:\\файлы_2гис_2023\\Астана\\Астана_2022_975011.csv',
  'D:\\файлы_2гис_2023\\Астана\\Астана_2023_975039.csv'],
 ['D:\\файлы_2гис_2023\\СПБ\\СПБ_2020_974783.csv',
  'D:\\файлы_2гис_2023\\СПБ\\СПБ_2021_974784.csv',
  'D:\\файлы_2гис_2023\\СПБ\\СПБ_2022_974785.csv',
  'D:\\файлы_2гис_2023\\СПБ\\СПБ_2023_974790.csv'],
 ['D:\\файлы_2гис_2023\\Ташкент\\ТАШКЕНТ_2020_974993.csv',
  'D:\\файлы_2гис_2023\\Ташкент\\ТАШКЕНТ_2021_974999.csv',
  'D:\\файлы_2гис_2023\\Ташкент\\ТАШКЕНТ_2022_975003.csv',
  'D:\\файлы_2гис_2023\\Ташкент\\ТАШКЕНТ_2023_975004.csv'],
 ['D:\\файлы_2гис_2023\\Новосибирск\\Новосибирск_2020_974791.csv',
  'D:\\файлы_2гис_2023\\Новосибирск\\Новосибирск_2021_974792.csv',
  'D:\\файлы_2гис_2023\\Новосибирск\\Новосибирск_2022_974795.csv',
  'D:\\файлы_2гис_2023\\Новосибирск\\Новосибирск_2023_974796.csv'],
 ['D:\\файлы_2гис_2023\\Алматы\\Алматы_2020_974797.csv',
  'D:\\файлы_2гис_2023\\Алматы\\Алматы_2021_974798.csv',
  'D:\\файлы_2гис_2023\\Алматы\\Алматы_2022_974799.csv',
  'D:\\файлы_2гис_2023\\Алматы\\Алматы_2023_974800.csv'],
 ['D:\\файлы_2гис_2023\\Москва\\Москва_2021_970748.csv',
  'D:\\файлы_2гис_2023\\Москва\\Москва_2022_970753.csv',
  'D:\\файлы_2гис_2023\\Москва\\Москва_2023_970754.csv',
  'D:\\файлы_2гис_2023\\Москва\\Москва_2020_969792.csv'],
 ['D:\\файлы_2гис_2023\\ОАЭ\\ОАЭ_2020_975082.csv',
  'D:\\файлы_2гис_2023\\ОАЭ\\ОАЭ_2021_975095.csv',
  'D:\\файлы_2гис_2023\\ОАЭ\\ОАЭ_2022_975102.csv',
  'D:\\файлы_2гис_2023\\ОАЭ\\ОАЭ_2023_975103.csv'],
 ['D:\\файлы_2гис_2023\\Бишкек\\БИШКЕК_2020_975045.csv',
  'D:\\файлы_2гис_2023\\Бишкек\\БИШКЕК_2021_975053.csv',
  'D:\\файлы_2гис_2023\\Бишкек\\БИШКЕК_2022_975062.csv',
  'D:\\файлы_2гис_2023\\Бишкек\\БИШКЕК_2023_975070.csv'],
 ['D:\\файлы_2гис_2023\\Владивосток\\Владивосток_2020_975628.csv',
  'D:\\файлы_2гис_2023\\Владивосток\\Владивосток_2021_975631.csv',
  'D:\\файлы_2гис_2023\\Владивосток\\Владивосток_2022_975648.csv',
  'D:\\файлы_2гис_2023\\Владивосток\\Владивосток_2023_975652.csv']]

In [3]:
from itertools import chain
files_list_chain = list(chain(*files_list))

In [13]:
print('Ready:', str(datetime.datetime.now())[:-7])

Ready: 2023-03-25 02:00:56
